In [1]:
from bs4 import BeautifulSoup
import requests
import random
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.safari.service import Service



In [2]:
url = 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html')

In [3]:
def generate_links():
    prefix = 'https://www.tapology.com'
    p_links = []
    base_link = soup.find('span', class_='last').find('a')['href']
    
    # Extract the current page number
    p_number = re.search(r"page=(\d+)", base_link)

    current_page = int(p_number.group(1))
    
    # Generate links by decrementing the page number
    while current_page > 0:
        new_link = re.sub(r"page=\d+", f"page={current_page}",base_link)
        full_link = prefix + new_link
        p_links.append(full_link)
        current_page -= 1
    
    return p_links

# Example usage
all_links = generate_links()
all_links


['https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=12&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=11&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=10&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=9&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=8&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=7&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=6&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=5&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=4&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=3&ranking=5058',
 'https://www.tapology.com/rankings/5058-2023-mma-fighter-of-the-year?page=2&

In [4]:
def fighter_stats():
    fighter_page_list = []
    for page in all_links:

        f_response = requests.get(page)
        f_soup = BeautifulSoup(f_response.text, 'html')

        divs = f_soup.find_all('div', class_='rankingItemsItemRow name')    
        prefix = 'https://www.tapology.com'

        for div in divs:
            a_tag = div.find('h1', class_='name').find('a')

            if a_tag:
                full_link = prefix + a_tag['href']
                fighter_page_list.append(full_link)
    
    return fighter_page_list

fighter_links = fighter_stats()

In [12]:
# divs = soup.find_all('div', class_='rankingItemsItemRow name')

# fighter_page_list = []
# prefix = 'https://www.tapology.com'

# for div in divs:
#     a_tag = div.find('h1', class_='name').find('a')

#     if a_tag:
#         full_link = prefix + a_tag['href']
#         fighter_page_list.append(full_link)

In [ ]:
# fighter_list = []

# for fighter in fighter_links:
#     res = requests.get(fighter)
#     content = BeautifulSoup(res.text, 'html')
#     def safe_get_text(tag, key):
#         result = tag.find('strong', text=key)
#         return result.find_next_sibling('span').text.strip() if result and result.find_next_sibling('span') else 'N/A'
    
#     name = safe_get_text(content, 'Name:')
#     nickname = safe_get_text(content, 'Nickname:')
#     record = safe_get_text(content, 'Pro MMA Record:')
#     earnings = safe_get_text(content, 'Career Disclosed Earnings:')
#     height = safe_get_text(content, 'Height:')
#     reach = safe_get_text(content, '| Reach:')
#     weight_class = safe_get_text(content, 'Weight Class:')
#     dob = safe_get_text(content, 'Age & Date of Birth:')
#     last_weight = safe_get_text(content, '| Last Weigh-In:')

#     block = f'Name:{name}, Nickname:{nickname}, Dob: {dob}, Record:{record}, Earnings:{earnings}, Height:{height}, Reach:{reach}, Weight:{last_weight}, Weight Class:{weight_class}'

#     fighter_list.append(block)

#     countdown = random.randint(2,10)
#     time.sleep(countdown)

In [5]:
# Function to safely get text from a BeautifulSoup tag
def safe_get_text(tag, key):
    result = tag.find('strong', text=key)
    return result.find_next_sibling('span').text.strip() if result and result.find_next_sibling('span') else 'N/A'

# Initialize Safari WebDriver
driver = webdriver.Safari()

fighter_list = []

for fighter in fighter_links:
    # Navigate to the fighter's page
    driver.get(fighter)
    time.sleep(3)  # Allow some time for the page to load

    # Get the page source and parse it with BeautifulSoup
    page_source = driver.page_source
    content = BeautifulSoup(page_source, 'html.parser')

    # Extract stats elements from website with safe_get_text function
    name = safe_get_text(content, 'Name:')
    if name == 'N/A':
        name = safe_get_text(content, 'Given Name:')
    nickname = safe_get_text(content, 'Nickname:')
    record = safe_get_text(content, 'Pro MMA Record:')
    earnings = safe_get_text(content, 'Career Disclosed Earnings:')
    height = safe_get_text(content, 'Height:')
    reach = safe_get_text(content, '| Reach:')
    weight_class = safe_get_text(content, 'Weight Class:')
    dob = safe_get_text(content, '| Date of Birth:')
    last_weight = safe_get_text(content, '| Last Weigh-In:')

    # Create information block of currently selected fighter and append it to the fighter_list
    block = f'Name: {name}, Nickname: {nickname}, DoB: {dob}, Record: {record}, Earnings: {earnings}, Height: {height}, Reach: {reach}, Weight: {last_weight}, Weight Class: {weight_class}'
    
    fighter_list.append([block])

    # Delay time randomly to act less like a bot
    time.sleep(random.randint(0, 7))

# Close WebDriver
driver.quit()


/var/folders/gy/7gnhtsnj5cg6_3cc6g04bmv40000gn/T/ipykernel_8862/886074945.py:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  result = tag.find('strong', text=key)


In [6]:
fighter_list

[['Name: Екатерина Шакалова, Nickname: N/A, DoB: 1997.08.31, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (158cm), Reach: 62.6" (159cm), Weight: 144.2 lbs, Weight Class: Featherweight'],
 ['Name: Emily Marisa Ducote, Nickname: Gordinha, DoB: 1994.01.01, Record: 13-9-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (157cm), Reach: 63.0" (160cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Fabacary Diatta, Nickname: N/A, DoB: 1996.10.16, Record: 9-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 70.5" (179cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Dumitru Girlean, Nickname: N/A, DoB: 1994.11.07, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: N/A, Weight: 154.3 lbs, Weight Class: Lightweight'],
 ['Name: Ana Talita de Oliveira Alencar, Nickname: Problem Child, DoB: 1990.10.17, Record: 5-0-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'1" (155cm), Reach: 58.5" (149cm), Weight: 116.0 lb

In [1]:
df = [['Name: Екатерина Шакалова, Nickname: N/A, DoB: 1997.08.31, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (158cm), Reach: 62.6" (159cm), Weight: 144.2 lbs, Weight Class: Featherweight'],
 ['Name: Emily Marisa Ducote, Nickname: Gordinha, DoB: 1994.01.01, Record: 13-9-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (157cm), Reach: 63.0" (160cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Fabacary Diatta, Nickname: N/A, DoB: 1996.10.16, Record: 9-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 70.5" (179cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Dumitru Girlean, Nickname: N/A, DoB: 1994.11.07, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: N/A, Weight: 154.3 lbs, Weight Class: Lightweight'],
 ['Name: Ana Talita de Oliveira Alencar, Nickname: Problem Child, DoB: 1990.10.17, Record: 5-0-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'1" (155cm), Reach: 58.5" (149cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Michael Joseph Perry, Nickname: Platinum, DoB: 1991.09.15, Record: 14-8-0 (Win-Loss-Draw), Earnings: $744,000 USD, Height: 5\'10" (178cm), Reach: 71.0" (180cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Diana Irena Belbiţă, Nickname: The Warrior Princess, DoB: 1996.06.26, Record: 15-9-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Jamal Pogues, Nickname: The Stormtrooper, DoB: 1995.12.01, Record: 11-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 77.0" (196cm), Weight: 265.5 lbs, Weight Class: Heavyweight'],
 ['Name: Elise Reed, Nickname: N/A, DoB: 1992.12.05, Record: 7-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'3" (160cm), Reach: 63.0" (160cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Dan Argueta, Nickname: The Determined, DoB: 1993.08.13, Record: 9-2-0, 2 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Nong Stamp, Nickname: N/A, DoB: 1997.09.27, Record: 11-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (157cm), Reach: 63.0" (160cm), Weight: 114.1 lbs, Weight Class: Strawweight'],
 ['Name: Carlos Hernandez, Nickname: N/A, DoB: 1993.10.25, Record: 9-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 67.0" (170cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: 田丸 辰, Tamaru Toki, Nickname: N/A, DoB: 2002.02.06, Record: N/A, Earnings: $0 USD, Height: 5\'4" (163cm), Reach: N/A, Weight: N/A, Weight Class: Flyweight'],
 ['Name: Marc Diakiese, Nickname: Bonecrusher, DoB: 1993.03.16, Record: 17-7-0 (Win-Loss-Draw), Earnings: $214,000 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: 吉成名高, Yoshinari Nadaka, Nickname: N/A, DoB: 2001.01.08, Record: N/A, Earnings: $0 USD, Height: 5\'4" (163cm), Reach: N/A, Weight: N/A, Weight Class: Atomweight'],
 ['Name: Luana Carolina Carvalho de Souza, Nickname: Dread, DoB: 1993.06.11, Record: 10-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 69.0" (175cm), Weight: 128.0 lbs, Weight Class: Flyweight'],
 ['Name: Davion Robby Franklin, Nickname: All Day, DoB: 1994.06.21, Record: 6-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 80.0" (203cm), Weight: 265.0 lbs, Weight Class: Heavyweight'],
 ['Name: Natalia Yuryevna Kuziutina, Nickname: Kuzya, DoB: 1989.05.08, Record: 5-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'3" (160cm), Reach: 64.5" (164cm), Weight: 115.4 lbs, Weight Class: Strawweight'],
 ['Name: Mitchell McKee, Nickname: Merciless, DoB: 1997.12.24, Record: 6-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 66.0" (168cm), Weight: 135.8 lbs, Weight Class: Bantamweight'],
 ['Name: Johnathan Thomas Dodson III, Nickname: The Magician, DoB: 1984.09.26, Record: 24-14-0 (Win-Loss-Draw), Earnings: $642,000 USD, Height: 5\'3" (161cm), Reach: 66.0" (168cm), Weight: 135.4 lbs, Weight Class: Flyweight'],
 ['Name: Kyle Driscoll, Nickname: N/A, DoB: 1994.09.23, Record: 14-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 72.0" (183cm), Weight: 155.7 lbs, Weight Class: Lightweight'],
 ['Name: Lucero Acosta, Nickname: La Loba, DoB: 1994.08.23, Record: 5-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 69.0" (175cm), Weight: 124.0 lbs, Weight Class: Flyweight'],
 ['Name: James Hendin, Nickname: N/A, DoB: 1997.11.12, Record: 9-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 71.0" (180cm), Weight: 145.7 lbs, Weight Class: Featherweight'],
 ['Name: Renato Valente Alves, Nickname: Índio, DoB: 1992.08.02, Record: 10-6-0 (Win-Loss-Draw), Earnings: $2,400 USD, Height: 6\'1" (185cm), Reach: 73.5" (187cm), Weight: 184.8 lbs, Weight Class: Middleweight'],
 ['Name: Jaclyn Cheree Cataline, Nickname: The Hybrid, DoB: 1988.05.27, Record: 4-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 65.5" (166cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Sabina Mazo Isaza, Nickname: Colombian Queen, DoB: 1997.03.25, Record: 10-4-0 (Win-Loss-Draw), Earnings: $44,000 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Cody Stamann, Nickname: The Spartan, DoB: 1989.11.09, Record: 21-6-1 (Win-Loss-Draw), Earnings: $337,000 USD, Height: 5\'6" (168cm), Reach: 64.5" (164cm), Weight: 139.0 lbs, Weight Class: Bantamweight'],
 ['Name: Diego Lopes, Nickname: N/A, DoB: N/A, Record: N/A, Earnings: $0 USD, Height: N/A, Reach: N/A, Weight: N/A, Weight Class: Featherweight'],
 ['Name: Jamey-Lyn Horth, Nickname: N/A, DoB: 1990.03.30, Record: 6-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 66.0" (168cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: 최두호, Choi Doo Ho, Nickname: The Korean Superboy, DoB: 1991.04.06, Record: 14-4-1 (Win-Loss-Draw), Earnings: $274,000 USD, Height: 5\'10" (178cm), Reach: 70.0" (178cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Rosemary Brito Conceição, Nickname: N/A, DoB: 1997.05.13, Record: 7-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'5" (165cm), Reach: N/A, Weight: 114.8 lbs, Weight Class: Strawweight'],
 ['Name: Лоик Раджабов, Nickname: The Tajik Tank, DoB: 1990.09.17, Record: 18-5-1 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'11" (180cm), Reach: 69.0" (175cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: 정찬성, Jung Chan Sung, Nickname: The Korean Zombie, DoB: 1987.03.17, Record: 17-8-0 (Win-Loss-Draw), Earnings: $574,000 USD, Height: 5\'7" (171cm), Reach: 72.0" (183cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Alfred Walker, Nickname: Freight Train, DoB: 1987.04.03, Record: 12-7-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 70.5" (179cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: 宋克南, Song Kenan, Nickname: The Assassin, DoB: 1990.03.10, Record: 21-8-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'0" (182cm), Reach: 71.5" (182cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Devin Smyth, Nickname: The Prodigy, DoB: 1995.07.21, Record: 10-4-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 169.4 lbs, Weight Class: Welterweight'],
 ['Name: Evelyn Martins, Nickname: Spartan, DoB: 2002.09.07, Record: 5-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 67.5" (171cm), Weight: 147.0 lbs, Weight Class: Featherweight'],
 ['Name: Neiman Gracie, Nickname: N/A, DoB: 1988.12.12, Record: 12-5-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 170.2 lbs, Weight Class: Welterweight'],
 ['Name: Kamarudeen Usman, Nickname: Nigerian Nightmare, DoB: 1987.05.11, Record: 20-4-0 (Win-Loss-Draw), Earnings: $1,794,000 USD, Height: 6\'0" (183cm), Reach: 76.0" (193cm), Weight: 184.5 lbs, Weight Class: Middleweight'],
 ['Name: Garry Lee Tonon, Nickname: The Lion Killer, DoB: 1991.09.12, Record: 9-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 69.0" (175cm), Weight: N/A, Weight Class: Lightweight'],
 ['Name: Берхамов Мухамед, Nickname: Cherkes, DoB: 1994.01.17, Record: 16-2-0, 1 NC (Win-Loss-Draw), Earnings: $80,000 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 178.4 lbs, Weight Class: Middleweight'],
 ['Name: အောင်လအန်ဆန်း, Nickname: The Burmese Python, DoB: 1985.05.21, Record: 30-13-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (186cm), Reach: N/A, Weight: 219.2 lbs, Weight Class: Light Heavyweight'],
 ['Name: Randi Field, Nickname: Rose City Phoenix, DoB: 1990.04.25, Record: 4-1-0 (Win-Loss-Draw), Earnings: $4,000 USD, Height: 5\'3" (160cm), Reach: 62.0" (157cm), Weight: 120.0 lbs, Weight Class: Flyweight'],
 ['Name: Cody Law, Nickname: N/A, DoB: 1995.03.24, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 68.5" (174cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Crystal Vanessa Demopoulos, Nickname: Lil Monster, DoB: 1988.09.22, Record: 11-5-0 (Win-Loss-Draw), Earnings: $74,000 USD, Height: 5\'2" (157cm), Reach: 59.5" (151cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Marouan Ioannis Bachar, Nickname: N/A, DoB: N/A, Record: 9-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 72.0" (183cm), Weight: 169.4 lbs, Weight Class: Welterweight'],
 ['Name: Carlos Diego Ferreira, Nickname: N/A, DoB: N/A, Record: 2-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: N/A, Reach: N/A, Weight: N/A, Weight Class: Lightweight'],
 ['Name: Norbert Novenyi Jr., Nickname: Magic, DoB: 1999.10.06, Record: 7-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: N/A, Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Bruna Emanuele Brasil, Nickname: The Special One, DoB: 1993.09.07, Record: 9-4-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 65.5" (166cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Geraldo Coelho de Lima Neto, Nickname: Luan Santana, DoB: 1990.12.09, Record: 18-7-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 72.0" (183cm), Weight: 169.6 lbs, Weight Class: Welterweight'],
 ['Name: Shajidul Haque, Nickname: Superman, DoB: 1990.10.29, Record: 16-5-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'5" (165cm), Reach: N/A, Weight: 124.4 lbs, Weight Class: Flyweight'],
 ['Name: Isaiah Hokit, Nickname: The Central Valley Gangster, DoB: 1996.05.07, Record: 3-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 69.0" (175cm), Weight: 145.6 lbs, Weight Class: Featherweight'],
 ['Name: Valeriu Mircea, Nickname: The Solitary Wolf, DoB: 1993.10.05, Record: 30-9-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 72.4" (184cm), Weight: 154.5 lbs, Weight Class: Lightweight'],
 ['Name: Waldo Cortes-Acosta, Nickname: Salsa Boy, DoB: 1991.10.03, Record: 12-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'4" (193cm), Reach: 78.0" (198cm), Weight: 262.0 lbs, Weight Class: Heavyweight'],
 ['Name: Abdul Razak Alhassan, Nickname: Judo Thunder, DoB: 1985.08.11, Record: 12-6-0 (Win-Loss-Draw), Earnings: $240,000 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 184.5 lbs, Weight Class: Middleweight'],
 ['Name: Диана Авсарагова, Nickname: Pantera, DoB: 1998.08.07, Record: 6-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (169cm), Reach: 65.5" (166cm), Weight: 127.2 lbs, Weight Class: Flyweight'],
 ['Name: Josuely Eduarda de Moura, Nickname: Ronda, DoB: 1994.02.28, Record: 10-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (167cm), Reach: 66.0" (168cm), Weight: 119.5 lbs, Weight Class: Strawweight'],
 ['Name: Robert Seronio III, Nickname: The Humble Warrior, DoB: 1998.05.21, Record: 4-0-0 (Win-Loss-Draw), Earnings: $7,000 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 138.0 lbs, Weight Class: Bantamweight'],
 ['Name: Marcin Prachnio, Nickname: N/A, DoB: 1988.07.14, Record: 17-7-0 (Win-Loss-Draw), Earnings: $75,000 USD, Height: 6\'3" (191cm), Reach: 74.0" (188cm), Weight: 204.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Eduardo Henrique da Silva, Nickname: Chapolin, DoB: 1995.11.23, Record: 12-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: N/A, Reach: N/A, Weight: 124.0 lbs, Weight Class: Flyweight'],
 ['Name: Tyrell Fortune, Nickname: N/A, DoB: 1990.07.04, Record: 13-2-0, 2 NC (Win-Loss-Draw), Earnings: $75,000 USD, Height: 6\'2" (188cm), Reach: 77.0" (196cm), Weight: 259.6 lbs, Weight Class: Heavyweight'],
 ['Name: Trevin Dewayne Giles, Nickname: The Problem, DoB: 1992.08.06, Record: 16-6-0 (Win-Loss-Draw), Earnings: $147,000 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Andrzej Grzebyk, Nickname: Double Champ, DoB: 1990.11.25, Record: 21-6-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (184cm), Reach: 76.0" (193cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: 伊澤星花, Nickname: N/A, DoB: 1997.11.01, Record: 13-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (157cm), Reach: 64.0" (163cm), Weight: 107.0 lbs, Weight Class: Atomweight'],
 ['Name: Charlie Campbell, Nickname: The Cannibal, DoB: 1995.07.03, Record: 9-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'0" (183cm), Reach: 72.5" (184cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Kyle Crutchmer, Nickname: N/A, DoB: 1993.05.05, Record: 10-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'9" (175cm), Reach: 68.0" (173cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Themba Takura Lawrence Gorimbo, Nickname: The Answer, DoB: 1991.01.23, Record: 13-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 77.0" (196cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Luana Munize Barbosa Pinheiro, Nickname: N/A, DoB: 1993.11.18, Record: 11-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'4" (163cm), Reach: 62.0" (157cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Matheus Mattos Mendes, Nickname: Adamas, DoB: 1992.06.06, Record: 14-2-1 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'5" (165cm), Reach: 67.0" (170cm), Weight: 135.2 lbs, Weight Class: Bantamweight'],
 ['Name: Arkadiusz Wrzosek, Nickname: Hightower, DoB: 1992.06.03, Record: 5-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'7" (201cm), Reach: 82.7" (210cm), Weight: 241.2 lbs, Weight Class: Heavyweight'],
 ['Name: Шараф Давлатмуродов, Nickname: Wolfhound, DoB: 1992.04.02, Record: 19-5-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 73.5" (187cm), Weight: 185.4 lbs, Weight Class: Middleweight'],
 ['Name: Ante Delija, Nickname: Walking Trouble, DoB: 1990.08.07, Record: 24-6-0 (Win-Loss-Draw), Earnings: $66,000 USD, Height: 6\'5" (196cm), Reach: 79.0" (201cm), Weight: 244.0 lbs, Weight Class: Heavyweight'],
 ['Name: Denise Kielholtz, Nickname: Miss Dynamite, DoB: 1989.03.30, Record: 8-5-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'3" (160cm), Reach: 66.0" (168cm), Weight: 124.0 lbs, Weight Class: Flyweight'],
 ['Name: Lucas Alexander, Nickname: The Lion, DoB: 1995.07.28, Record: 8-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 73.0" (185cm), Weight: 148.0 lbs, Weight Class: Featherweight'],
 ['Name: Blake Richard Bilder, Nickname: El Animal, DoB: 1990.07.12, Record: 8-2-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 68.5" (174cm), Weight: 144.5 lbs, Weight Class: Featherweight'],
 ['Name: Esteban Ribovics Marijan, Nickname: El Gringo, DoB: 1996.04.27, Record: 13-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 69.0" (175cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Jamie Patrick Mullarkey, Nickname: N/A, DoB: 1994.08.17, Record: 17-8-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Raul Gilberto Rosas Rios, Nickname: El Niño Problema, DoB: 2004.10.08, Record: 8-1-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'9" (175cm), Reach: 67.0" (170cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Claudio Da Silva Ribeiro, Nickname: N/A, DoB: 1992.07.17, Record: 11-5-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 77.0" (196cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Tyson Pedro, Nickname: N/A, DoB: 1991.09.17, Record: 10-5-0 (Win-Loss-Draw), Earnings: $160,000 USD, Height: 6\'3" (191cm), Reach: 79.0" (201cm), Weight: 205.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Melquizael Costa da Conceição, Nickname: Melk Cauthy, DoB: 1996.09.14, Record: 20-7-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Chris Gonzalez, Nickname: CG, DoB: 1991.07.29, Record: 8-3-0 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'11" (180cm), Reach: 73.0" (185cm), Weight: 155.4 lbs, Weight Class: Lightweight'],
 ['Name: Jared Gooden, Nickname: NiteTrain, DoB: 1993.12.05, Record: 23-9-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'0" (183cm), Reach: 77.0" (196cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Danni McCormack, Nickname: Mac, DoB: 1990.03.15, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'4" (163cm), Reach: 61.5" (156cm), Weight: 114.6 lbs, Weight Class: Strawweight'],
 ['Name: Joselyne Edwards Loboriel, Nickname: La Pantera, DoB: 1995.09.29, Record: 13-6-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Daniel Frunza, Nickname: Tigano, DoB: 1994.04.13, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: N/A, Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Jesus Santos Aguilar Rodriguez, Nickname: N/A, DoB: 1996.03.13, Record: 10-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'4" (163cm), Reach: 62.0" (157cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Сергей Билостенный, Nickname: N/A, DoB: 1995.10.23, Record: 12-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 75.0" (191cm), Weight: 248.4 lbs, Weight Class: Heavyweight'],
 ['Name: Tereza Bledá, Nickname: N/A, DoB: 2001.11.30, Record: 7-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 71.0" (180cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Darren Stewart, Nickname: The Dentist, DoB: 1990.12.30, Record: 16-10-0, 2 NC (Win-Loss-Draw), Earnings: $182,000 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 194.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Jake Matthews, Nickname: The Celtic Kid, DoB: 1994.08.19, Record: 20-7-0 (Win-Loss-Draw), Earnings: $172,000 USD, Height: 5\'11" (180cm), Reach: 73.0" (185cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Jakub Kaszuba, The Boar, Nickname: The Auditor, DoB: 1995.06.02, Record: 12-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 73.0" (185cm), Weight: 154.9 lbs, Weight Class: Lightweight'],
 ['Name: Joseph Solecki, Nickname: N/A, DoB: 1993.08.27, Record: 13-5-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'9" (175cm), Reach: 70.5" (179cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Nathaniel Wood, Nickname: The Prospect, DoB: 1993.08.05, Record: 19-6-0 (Win-Loss-Draw), Earnings: $124,000 USD, Height: 5\'6" (168cm), Reach: 69.0" (175cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Keri Anne Taylor-Melendez, Nickname: N/A, DoB: 1984.05.12, Record: 6-0-0 (Win-Loss-Draw), Earnings: $75,000 USD, Height: 5\'6" (168cm), Reach: 67.5" (171cm), Weight: 126.6 lbs, Weight Class: Flyweight'],
 ['Name: Азамат Бекоев, Nickname: King Pin, DoB: 1995.12.30, Record: 17-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 72.0" (183cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Roberto Hernandez, Nickname: Green Light, DoB: 1993.07.04, Record: 9-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: N/A, Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: 최승우, Choi Seung Woo, Nickname: Sting, DoB: 1992.11.03, Record: 11-6-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'0" (183cm), Reach: 74.5" (189cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Альберт Дураев, Nickname: Machete, DoB: 1989.01.05, Record: 16-5-0 (Win-Loss-Draw), Earnings: $5,000 USD, Height: 5\'11" (180cm), Reach: 75.0" (191cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Alexander Munoz, Nickname: N/A, DoB: 1990.02.27, Record: 7-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 72.0" (183cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Miles Johns, Nickname: Chapo, DoB: 1994.03.30, Record: 14-2-0, 1 NC (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'7" (170cm), Reach: 66.0" (168cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Karolline Rosa Cavedo, Nickname: N/A, DoB: 1994.12.30, Record: 17-6-0 (Win-Loss-Draw), Earnings: $78,000 USD, Height: 5\'5" (165cm), Reach: 67.5" (171cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Jefferson Nascimento, Nickname: Todynho, DoB: N/A, Record: 11-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: N/A, Weight: 154.6 lbs, Weight Class: Lightweight'],
 ['Name: Ange Loosa, Nickname: The Last Ninja, DoB: 1994.03.18, Record: 10-3-0, 1 NC (Win-Loss-Draw), Earnings: $24,000 USD, Height: 5\'10" (178cm), Reach: 74.0" (188cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Ignacio A Bahamondes Carabantes, Nickname: La Jaula, DoB: 1997.08.27, Record: 15-5-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'3" (191cm), Reach: 75.5" (192cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Trevor Peek, Nickname: N/A, DoB: 1995.01.09, Record: 9-2-0, 1 NC (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'9" (175cm), Reach: 70.0" (178cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Jordan Leavitt, Nickname: The Monkey King, DoB: 1995.06.02, Record: 11-3-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'9" (175cm), Reach: 71.0" (180cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Adam Fugitt, Nickname: N/A, DoB: 1989.01.12, Record: 9-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 77.0" (196cm), Weight: 170.8 lbs, Weight Class: Welterweight'],
 ['Name: Ashton Fletcher, Nickname: The Ghost, DoB: 1997.02.18, Record: 10-4-0 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'10" (178cm), Reach: 67.0" (170cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Elizeu Zaleski dos Santos, Nickname: Capoeira, DoB: 1986.11.12, Record: 24-8-1 (Win-Loss-Draw), Earnings: $194,000 USD, Height: 5\'11" (180cm), Reach: 73.0" (185cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Jeka Asparido Saragih, Nickname: Si Tendangan Maut, DoB: 1995.01.01, Record: 14-3-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'8" (172cm), Reach: 69.0" (175cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Kleidison De Sousa Rodrigues, Nickname: KR, DoB: 1995.10.22, Record: 8-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'5" (165cm), Reach: 67.0" (170cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Michał Materla, Nickname: Cipao, DoB: 1984.04.10, Record: 33-11-0 (Win-Loss-Draw), Earnings: $12,000 USD, Height: 6\'0" (183cm), Reach: 75.6" (192cm), Weight: 185.6 lbs, Weight Class: Middleweight'],
 ['Name: Gabriel Miranda, Nickname: Fly, DoB: 1990.03.25, Record: 17-6-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 71.0" (180cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Yanal Ashmouz, Nickname: Red Fox, DoB: 1995.03.31, Record: 7-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 68.0" (173cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Mamed Khalidov, Nickname: Cannibal, DoB: 1980.07.17, Record: 37-8-2 (Win-Loss-Draw), Earnings: $7,000 USD, Height: 6\'0" (183cm), Reach: 76.4" (194cm), Weight: 186.1 lbs, Weight Class: Middleweight'],
 ['Name: Yusuf Karakaya, Nickname: Baracuda, DoB: N/A, Record: 2-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 71.0" (180cm), Weight: 170.2 lbs, Weight Class: Welterweight'],
 ['Name: Rafael Ramos Estevam, Nickname: Macapá, DoB: 1996.08.10, Record: 12-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 128.0 lbs, Weight Class: Flyweight'],
 ['Name: Dakota Lili-Joa Ditcheva, Nickname: Dangerous, DoB: 1998.07.11, Record: 11-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 125.2 lbs, Weight Class: Flyweight'],
 ['Name: Talita Bernardo, Nickname: N/A, DoB: 1987.03.04, Record: 10-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'4" (163cm), Reach: 68.0" (173cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Ketlen Gabriela Da Silva Souza, Nickname: Esquentadinha, DoB: 1995.08.18, Record: 14-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'3" (161cm), Reach: 63.0" (160cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Taylor Lapilus, Nickname: Double Impact, DoB: 1992.04.08, Record: 19-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 73.0" (185cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Charles William Radtke, Nickname: Chuck Buffalo, DoB: 1990.07.09, Record: 9-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 72.0" (183cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Rayanne Amanda dos Santos, Nickname: N/A, DoB: 1995.06.08, Record: 14-7-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (157cm), Reach: 62.0" (157cm), Weight: 115.0 lbs, Weight Class: Strawweight'],
 ['Name: Joaquim Antônio Magalhães Silva, Nickname: Netto BJJ, DoB: 1989.02.05, Record: 13-5-0 (Win-Loss-Draw), Earnings: $172,000 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Brendan Loughnane, Nickname: N/A, DoB: 1989.12.05, Record: 28-5-0 (Win-Loss-Draw), Earnings: $220,000 USD, Height: 5\'9" (176cm), Reach: 71.0" (180cm), Weight: 145.8 lbs, Weight Class: Featherweight'],
 ['Name: Jordan Vucenic, Nickname: The Epidemic, DoB: 1996.03.02, Record: 12-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 72.0" (183cm), Weight: 145.8 lbs, Weight Class: Featherweight'],
 ['Name: Потеря Игорь, Nickname: Duelist, DoB: 1996.05.29, Record: 21-6-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'3" (190cm), Reach: 75.0" (191cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Zebaztian Kadestam, Nickname: The Bandit, DoB: 1990.09.27, Record: 15-7-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: N/A, Weight: 170.0 lbs, Weight Class: Middleweight'],
 ['Name: 傲日其楞, Nickname: The Mongolian Murderer, DoB: 1993.06.25, Record: 25-11-0, 1 NC (Win-Loss-Draw), Earnings: $74,000 USD, Height: 5\'7" (170cm), Reach: 69.3" (176cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: 井上瑞樹, Inoue Mizuki, Nickname: N/A, DoB: 1994.08.19, Record: 15-6-0 (Win-Loss-Draw), Earnings: $2,000 USD, Height: 5\'3" (160cm), Reach: 64.0" (163cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Hugo Cunha, Nickname: Silverback, DoB: 1993.02.04, Record: 8-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'4" (193cm), Reach: N/A, Weight: 258.8 lbs, Weight Class: Heavyweight'],
 ['Name: Montserrat Rendon, Nickname: Monster, DoB: 1989.03.16, Record: 6-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (172cm), Reach: 68.0" (173cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Parker Porter, Nickname: N/A, DoB: 1985.04.22, Record: 14-9-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 75.0" (191cm), Weight: 256.0 lbs, Weight Class: Heavyweight'],
 ['Name: Asaël Adjoudj, Nickname: The Sultan, DoB: 1999.01.16, Record: 8-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (182cm), Reach: 71.5" (182cm), Weight: 145.9 lbs, Weight Class: Featherweight'],
 ['Name: Jamall Emmers, Nickname: Pretty Boy, DoB: 1989.07.24, Record: 20-8-0 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'10" (178cm), Reach: 74.0" (188cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Brian Moore, Nickname: The Pikeman, DoB: 1987.08.13, Record: 16-10-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 145.4 lbs, Weight Class: Bantamweight'],
 ['Name: Josefine Lindgren Knutsson, Nickname: Thunder, DoB: 1996.01.16, Record: 7-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'3" (160cm), Reach: 60.0" (152cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Evan Elder, Nickname: The Phenom, DoB: 1997.04.11, Record: 8-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'10" (178cm), Reach: 71.0" (180cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Ode’ Omaani Osbourne, Nickname: The Jamaican Sensation, DoB: 1992.01.09, Record: 12-7-0, 1 NC (Win-Loss-Draw), Earnings: $88,000 USD, Height: 5\'7" (171cm), Reach: 73.0" (185cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: Kenny Mokhonoana, Nickname: Black Diamond, DoB: 1997.01.20, Record: 5-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Sean Woodson, Nickname: The Sniper, DoB: 1992.06.07, Record: 12-1-1 (Win-Loss-Draw), Earnings: $36,000 USD, Height: 6\'2" (189cm), Reach: 78.0" (198cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Zachary Taala Pauga, Nickname: The High Chief, DoB: 1988.02.25, Record: 7-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 76.0" (193cm), Weight: 241.6 lbs, Weight Class: Heavyweight'],
 ['Name: Garrett Armfield, Nickname: N/A, DoB: 1996.10.09, Record: 10-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 70.0" (178cm), Weight: 135.3 lbs, Weight Class: Bantamweight'],
 ['Name: Melsik Baghdasaryan, Nickname: The Gun, DoB: 1992.01.28, Record: 8-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'9" (175cm), Reach: 70.0" (178cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Cody Davis, Nickname: N/A, DoB: 1994.08.17, Record: 9-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'3" (160cm), Reach: 66.5" (169cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Alfie Davis, Nickname: The Axe Man, DoB: 1992.02.25, Record: 17-4-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 70.0" (178cm), Weight: 155.6 lbs, Weight Class: Lightweight'],
 ['Name: Саид Нурмагомедов, Nickname: N/A, DoB: 1992.04.05, Record: 18-3-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Gregory Felipe Santos Rodrigues, Nickname: Robocop, DoB: 1992.02.17, Record: 15-5-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 6\'3" (191cm), Reach: 75.0" (191cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Вугар Кярамов, Nickname: Vugi, DoB: 1992.02.20, Record: 19-5-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (176cm), Reach: 70.5" (179cm), Weight: 144.6 lbs, Weight Class: Featherweight'],
 ['Name: Jack Jenkins, Nickname: Phar, DoB: 1993.03.11, Record: 12-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 145.9 lbs, Weight Class: Featherweight'],
 ['Name: Bruna Ellen, Nickname: N/A, DoB: 1996.06.26, Record: 7-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'4" (163cm), Reach: 65.0" (165cm), Weight: 125.6 lbs, Weight Class: Flyweight'],
 ['Name: Robert Ruchała, Nickname: N/A, DoB: 1998.05.15, Record: 10-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 72.4" (184cm), Weight: 144.2 lbs, Weight Class: Featherweight'],
 ['Name: Rhys McKee, Nickname: Skeletor, DoB: 1995.09.10, Record: 13-6-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 78.0" (198cm), Weight: 170.3 lbs, Weight Class: Welterweight'],
 ['Name: Adrian Bartosiński, Nickname: Bartos, DoB: 1995.08.26, Record: 16-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (181cm), Reach: 71.7" (182cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Fernando Lucas, Nickname: N/A, DoB: N/A, Record: N/A, Earnings: $0 USD, Height: N/A, Reach: N/A, Weight: N/A, Weight Class: '],
 ['Name: Payton Talbott, Nickname: N/A, DoB: 1998.09.09, Record: 8-0-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'10" (178cm), Reach: 70.0" (178cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Timothy Carl Means, Nickname: The Dirty Bird, DoB: 1984.02.20, Record: 33-16-1, 1 NC (Win-Loss-Draw), Earnings: $586,000 USD, Height: 6\'2" (188cm), Reach: 75.0" (191cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Bryce Meredith, Nickname: Misfit, DoB: 1995.04.30, Record: 6-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 68.5" (174cm), Weight: 135.8 lbs, Weight Class: Bantamweight'],
 ['Name: JaCobi Jones, Nickname: Big Toe, DoB: 1996.05.24, Record: 6-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 72.0" (183cm), Weight: 155.2 lbs, Weight Class: Lightweight'],
 ['Name: Jake Hadley, Nickname: White Kong, DoB: 1996.08.03, Record: 10-3-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Jerrell Hodge, Nickname: Thunderkat, DoB: 1989.11.24, Record: 10-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 70.0" (178cm), Weight: 135.6 lbs, Weight Class: Bantamweight'],
 ['Name: Alessandro Costa, Nickname: Nono, DoB: 1996.01.28, Record: 14-4-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'4" (163cm), Reach: 67.0" (170cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Paweł Pawlak, Nickname: Plastinho, DoB: 1989.02.08, Record: 23-4-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 184.7 lbs, Weight Class: Middleweight'],
 ['Name: Mandy Böhm, Nickname: Monster, DoB: 1989.07.30, Record: 8-2-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 71.0" (180cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Alexandr Anatolyevich Romanov, Nickname: King Kong, DoB: 1990.12.11, Record: 17-3-0 (Win-Loss-Draw), Earnings: $86,000 USD, Height: 6\'2" (188cm), Reach: 75.0" (191cm), Weight: 265.0 lbs, Weight Class: Heavyweight'],
 ['Name: Eduardo José Neves, Nickname: Bebezao, DoB: 2000.03.16, Record: 8-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 78.5" (199cm), Weight: 264.2 lbs, Weight Class: Heavyweight'],
 ['Name: Miguel Felipe Bunes da Silva, Nickname: Felipinho, DoB: 1989.10.26, Record: 13-7-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 71.0" (180cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: The meal deal, Nickname: The Hillbilly Hammer, DoB: 1993.07.02, Record: 7-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 184.3 lbs, Weight Class: Middleweight'],
 ['Name: Caolán Loughran, Nickname: The Don, DoB: 1996.05.18, Record: 9-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 68.0" (173cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Yves Landu, Nickname: You Know, DoB: 1986.08.13, Record: 21-9-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 73.0" (185cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Sullivan Cauley, Nickname: Sully, DoB: 1996.05.15, Record: 6-1-0 (Win-Loss-Draw), Earnings: $4,000 USD, Height: 6\'3" (191cm), Reach: 74.5" (189cm), Weight: 205.6 lbs, Weight Class: Light Heavyweight'],
 ['Name: Gaston Andre Bolaños, Nickname: The Dreamkiller, DoB: 1992.09.14, Record: 7-4-0 (Win-Loss-Draw), Earnings: $87,000 USD, Height: 5\'7" (170cm), Reach: 69.0" (175cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Victor Altamirano Romano, Nickname: El Magnifico, DoB: 1991.01.23, Record: 12-4-0 (Win-Loss-Draw), Earnings: $70,000 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: George Hardwick, Nickname: N/A, DoB: 1996.12.06, Record: 12-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 72.0" (183cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Billy Goff, Nickname: N/A, DoB: 1998.06.18, Record: 9-3-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'10" (178cm), Reach: 72.5" (184cm), Weight: 170.3 lbs, Weight Class: Welterweight'],
 ['Name: Jordan Oliver, Nickname: That Dude, DoB: 1990.05.08, Record: 1-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 70.5" (179cm), Weight: 145.6 lbs, Weight Class: Featherweight'],
 ['Name: Jakob Nedoh, Nickname: The Gorilla, DoB: 1996.10.01, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 73.0" (185cm), Weight: 206.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Josh Hokit, Nickname: The Incredible Hok, DoB: 1997.11.12, Record: 1-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: N/A, Weight: 246.0 lbs, Weight Class: Heavyweight'],
 ['Name: Marcirley Alves da Silva, Nickname: Durin, DoB: 1999.07.22, Record: 12-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'4" (163cm), Reach: 68.0" (173cm), Weight: 135.2 lbs, Weight Class: Bantamweight'],
 ['Name: Mark Ewen, Nickname: N/A, DoB: N/A, Record: 5-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Марина Мохнаткина, Nickname: N/A, DoB: 1988.05.12, Record: 11-4-0 (Win-Loss-Draw), Earnings: $32,000 USD, Height: 5\'6" (168cm), Reach: 66.0" (168cm), Weight: 144.0 lbs, Weight Class: Featherweight'],
 ['Name: Sinead Kavanagh, Nickname: KO, DoB: 1986.02.01, Record: 9-6-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 145.8 lbs, Weight Class: Featherweight'],
 ['Name: Marina Alcalde Rodriguez, Nickname: N/A, DoB: 1987.04.29, Record: 17-4-2 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'6" (168cm), Reach: 65.0" (165cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Javier Garcia, Nickname: El Mariachi, DoB: 1992.07.03, Record: 10-5-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 69.0" (175cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Henry Corrales, Nickname: OK, DoB: 1986.08.11, Record: 21-8-0 (Win-Loss-Draw), Earnings: $186,000 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 155.7 lbs, Weight Class: Lightweight'],
 ['Name: Charles Anthony Johnson, Nickname: InnerG, DoB: 1991.01.10, Record: 15-6-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'9" (175cm), Reach: 70.0" (178cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: Zhakwarius Ya\'myl Jose Johnson, Nickname: Lobo Solitario, DoB: 1995.05.20, Record: 16-8-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 71.5" (182cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Khurshed Kakhorov, Nickname: Killer, DoB: 1992.02.20, Record: 12-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 68.0" (173cm), Weight: 134.6 lbs, Weight Class: Bantamweight'],
 ['Name: 渡辺華奈, Nickname: N/A, DoB: 1988.08.21, Record: 13-2-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (167cm), Reach: 66.0" (168cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: 菊入正行, Nickname: N/A, DoB: 1995.06.30, Record: 9-2-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (184cm), Reach: 73.5" (187cm), Weight: 169.0 lbs, Weight Class: Welterweight'],
 ['Name: Brent Primus, Nickname: N/A, DoB: 1985.04.12, Record: 13-3-0, 1 NC (Win-Loss-Draw), Earnings: $66,000 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 155.8 lbs, Weight Class: Lightweight'],
 ['Name: Otto Rodrigues dos Santos Lima, Nickname: N/A, DoB: 1988.03.28, Record: 14-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 62.0" (157cm), Weight: 145.2 lbs, Weight Class: Featherweight'],
 ['Name: Jason Marlon Jackson, Nickname: The Ass-Kicking Machine, DoB: 1990.10.30, Record: 18-4-0 (Win-Loss-Draw), Earnings: $5,000 USD, Height: 6\'1" (185cm), Reach: 78.5" (199cm), Weight: 182.1 lbs, Weight Class: Middleweight'],
 ['Name: Larissa Moreira Pacheco, Nickname: N/A, DoB: 1994.09.07, Record: 23-4-0 (Win-Loss-Draw), Earnings: $98,000 USD, Height: 5\'6" (168cm), Reach: 69.0" (175cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Alex Caceres, Nickname: Bruce Leeroy, DoB: 1988.06.20, Record: 21-15-0, 1 NC (Win-Loss-Draw), Earnings: $752,000 USD, Height: 5\'10" (178cm), Reach: 73.5" (187cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Довлетджан Ягшимуратов, Nickname: Wolfhound, DoB: 1989.05.22, Record: 22-7-1 (Win-Loss-Draw), Earnings: $140,000 USD, Height: 5\'11" (180cm), Reach: 72.0" (183cm), Weight: 205.6 lbs, Weight Class: Light Heavyweight'],
 ['Name: Гаджи Рабаданов, Nickname: N/A, DoB: 1993.07.01, Record: 21-4-2 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 71.0" (180cm), Weight: 155.8 lbs, Weight Class: Lightweight'],
 ['Name: Joshua Silveira, Nickname: Coninha, DoB: 1992.12.25, Record: 13-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'1" (185cm), Reach: 75.0" (191cm), Weight: 205.4 lbs, Weight Class: Light Heavyweight'],
 ['Name: Juan Archuleta, Nickname: Spaniard, DoB: 1987.09.13, Record: 29-5-0 (Win-Loss-Draw), Earnings: $125,750 USD, Height: 5\'7" (170cm), Reach: 69.0" (175cm), Weight: 134.0 lbs, Weight Class: Bantamweight'],
 ['Name: Melissa Tonya Mullins, Nickname: No Mess, DoB: 1991.07.23, Record: 6-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 138.0 lbs, Weight Class: Bantamweight'],
 ['Name: Ciaran Clarke, Nickname: N/A, DoB: 1995.05.25, Record: 9-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 66.0" (168cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Philip De Fries, Nickname: N/A, DoB: 1986.04.21, Record: 24-6-0, 1 NC (Win-Loss-Draw), Earnings: $15,000 USD, Height: 6\'4" (194cm), Reach: 79.0" (201cm), Weight: 264.6 lbs, Weight Class: Heavyweight'],
 ['Name: Sam Hughes, Nickname: Sampage, DoB: 1992.06.28, Record: 8-6-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'5" (165cm), Reach: 64.5" (164cm), Weight: 115.0 lbs, Weight Class: Strawweight'],
 ['Name: Andre Riley Givens-Fili, Nickname: Touchy, DoB: 1990.06.25, Record: 23-11-0, 1 NC (Win-Loss-Draw), Earnings: $500,200 USD, Height: 5\'11" (180cm), Reach: 74.0" (188cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Theo Haig, Nickname: N/A, DoB: 1996.08.23, Record: 3-0-0 (Win-Loss-Draw), Earnings: $4,000 USD, Height: 6\'2" (188cm), Reach: 76.0" (193cm), Weight: 184.2 lbs, Weight Class: Middleweight'],
 ['Name: Bradley Devin Katona, Nickname: Superman, DoB: 1991.12.19, Record: 13-3-0 (Win-Loss-Draw), Earnings: $80,000 USD, Height: 5\'6" (168cm), Reach: 64.5" (164cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Christian Leroy Duncan, Nickname: CLD, DoB: 1995.07.24, Record: 10-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 79.0" (201cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Richie Smullen, Nickname: N/A, DoB: 1991.08.17, Record: 13-3-1 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'9" (175cm), Reach: 71.0" (180cm), Weight: 145.0 lbs, Weight Class: Lightweight'],
 ['Name: Jacqueline Cavalcanti, Nickname: N/A, DoB: 1997.08.29, Record: 6-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 139.0 lbs, Weight Class: Featherweight'],
 ['Name: Хасан Магомедшарипов, Nickname: N/A, DoB: 2000.10.06, Record: 9-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 71.0" (180cm), Weight: 145.6 lbs, Weight Class: Featherweight'],
 ['Name: Sarvarjon Khamidov, Nickname: Sarvar, DoB: 1997.04.13, Record: 15-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 68.0" (173cm), Weight: 140.6 lbs, Weight Class: Featherweight'],
 ['Name: 鈴木千裕, Suzuki Chihiro, Nickname: N/A, DoB: 1999.05.14, Record: 13-3-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 69.7" (177cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Daniel Gustavo Santos, Nickname: Willycat, DoB: 1995.03.12, Record: 11-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 67.0" (170cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Clayton Carpenter, Nickname: N/A, DoB: 1996.06.04, Record: 7-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 66.0" (168cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: JJ Aldrich, Nickname: N/A, DoB: 1992.09.29, Record: 13-7-0 (Win-Loss-Draw), Earnings: $67,250 USD, Height: 5\'5" (165cm), Reach: 67.5" (171cm), Weight: 125.3 lbs, Weight Class: Flyweight'],
 ['Name: Lorenz W. Larkin, Nickname: The Monsoon, DoB: 1986.09.03, Record: 25-8-0, 2 NC (Win-Loss-Draw), Earnings: $684,975 USD, Height: 5\'11" (181cm), Reach: 72.5" (184cm), Weight: 170.6 lbs, Weight Class: Welterweight'],
 ['Name: 이정영, Lee Jung Young, Nickname: The Korean Tiger, DoB: 1995.11.13, Record: 11-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Athlone Dobson, Nickname: N/A, DoB: 1992.01.18, Record: 7-3-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 76.0" (193cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Martin Buday, Nickname: Badys, DoB: 1991.11.11, Record: 13-2-0 (Win-Loss-Draw), Earnings: $24,000 USD, Height: 6\'4" (193cm), Reach: 77.0" (196cm), Weight: 264.5 lbs, Weight Class: Heavyweight'],
 ['Name: Wladimir Gouvea Massi Da Cunha, Nickname: Blanka, DoB: 1992.11.17, Record: 2-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 264.0 lbs, Weight Class: Heavyweight'],
 ['Name: Денис Гольцов, Nickname: The Russian Bogatyr, DoB: 1990.06.10, Record: 33-8-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 6\'5" (196cm), Reach: 78.0" (198cm), Weight: 245.4 lbs, Weight Class: Heavyweight'],
 ['Name: Jaqueline de Moraes Amorim Ramos, Nickname: Jacque, DoB: 1995.06.24, Record: 8-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'3" (160cm), Reach: 68.0" (173cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Mick Stanton, Nickname: The Huyton Hammer, DoB: 1987.01.27, Record: 13-8-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 74.0" (188cm), Weight: 185.8 lbs, Weight Class: Middleweight'],
 ['Name: Clay Jordan Collard, Nickname: Cassius, DoB: 1993.03.10, Record: 25-12-0, 1 NC (Win-Loss-Draw), Earnings: $62,000 USD, Height: 5\'11" (180cm), Reach: 73.0" (185cm), Weight: 155.6 lbs, Weight Class: Lightweight'],
 ['Name: Sadibou Sy, Nickname: The Swedish Denzel Washington, DoB: 1986.11.27, Record: 16-8-2, 1 NC (Win-Loss-Draw), Earnings: $177,000 USD, Height: 6\'3" (191cm), Reach: 80.0" (203cm), Weight: 205.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Cris Lencioni, Nickname: Sunshine, DoB: 1994.07.28, Record: 11-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 71.0" (180cm), Weight: 152.4 lbs, Weight Class: Featherweight'],
 ['Name: Sergio Daniel Cossio Dominguez, Nickname: Drako, DoB: N/A, Record: 26-9-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Anatoly Malykhin, Nickname: Spartak, DoB: 1988.01.11, Record: 14-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: N/A, Weight: N/A, Weight Class: Light Heavyweight'],
 ['Name: 강경호, Kang Kyung Ho, Nickname: Mr. Perfect, DoB: 1987.09.09, Record: 19-10-0, 1 NC (Win-Loss-Draw), Earnings: $118,000 USD, Height: 5\'9" (176cm), Reach: 73.0" (185cm), Weight: 138.0 lbs, Weight Class: Bantamweight'],
 ['Name: Christopher Gutierrez, Nickname: El Guapo, DoB: 1991.04.22, Record: 20-5-2 (Win-Loss-Draw), Earnings: $172,000 USD, Height: 5\'9" (175cm), Reach: 67.0" (170cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Dario Bellandi, Nickname: Neanderthal, DoB: N/A, Record: 7-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (190cm), Reach: N/A, Weight: 184.9 lbs, Weight Class: Middleweight'],
 ['Name: Killys Mota, Nickname: N/A, DoB: 1990.11.03, Record: 15-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 71.0" (180cm), Weight: 160.0 lbs, Weight Class: Lightweight'],
 ['Name: Tanner Boser, Nickname: The Bulldozer, DoB: 1991.08.02, Record: 21-10-1 (Win-Loss-Draw), Earnings: $74,000 USD, Height: 6\'2" (188cm), Reach: 75.5" (192cm), Weight: 204.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Thiago Henrique Moisés, Nickname: N/A, DoB: 1995.03.23, Record: 18-7-0 (Win-Loss-Draw), Earnings: $124,000 USD, Height: 5\'9" (175cm), Reach: 70.5" (179cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Liam Gittins, Nickname: Nightmare, DoB: N/A, Record: 13-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: N/A, Weight: 134.7 lbs, Weight Class: Bantamweight'],
 ['Name: John Sweeney, Nickname: Hollywood, DoB: 1995.04.06, Record: 13-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Daniel Pineda, Nickname: The Pit, DoB: 1985.08.06, Record: 28-15-0, 3 NC (Win-Loss-Draw), Earnings: $326,000 USD, Height: 5\'7" (170cm), Reach: 69.0" (175cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Gabriel Alves Braga, Nickname: N/A, DoB: 1998.04.24, Record: 13-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 72.0" (183cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Sharabutdin Magomedov (Шарабутдин Магомедов), Nickname: Bullet, DoB: 1994.05.16, Record: 12-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 73.0" (185cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Randall Joseph Brown, Nickname: Rudeboy, DoB: 1990.07.08, Record: 19-5-0 (Win-Loss-Draw), Earnings: $112,000 USD, Height: 6\'3" (191cm), Reach: 78.0" (198cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Michał Oleksiejczuk, Nickname: Hussar, DoB: 1995.02.22, Record: 19-8-0, 1 NC (Win-Loss-Draw), Earnings: $174,000 USD, Height: 6\'0" (184cm), Reach: 74.0" (188cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Bradley Kaipo Sarbida Tavares, Nickname: N/A, DoB: 1987.12.21, Record: 20-9-0 (Win-Loss-Draw), Earnings: $497,000 USD, Height: 6\'1" (185cm), Reach: 74.0" (188cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Fernando Padilla, Nickname: El Valiente, DoB: 1996.12.15, Record: 16-5-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'1" (186cm), Reach: 76.0" (193cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Азат Максум, Nickname: Qazaq, DoB: 1995.02.02, Record: 15-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Халид Муртазалиев, Nickname: N/A, DoB: 1993.07.15, Record: 17-3-0 (Win-Loss-Draw), Earnings: $33,000 USD, Height: 6\'0" (183cm), Reach: 72.5" (184cm), Weight: 186.8 lbs, Weight Class: Middleweight'],
 ['Name: Armen Karlenovich Petrosyan, Nickname: Superman, DoB: 1990.11.03, Record: 8-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 71.0" (180cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Timothy Johnson, Nickname: N/A, DoB: 1985.03.13, Record: 17-9-0 (Win-Loss-Draw), Earnings: $141,000 USD, Height: 6\'3" (191cm), Reach: 78.0" (198cm), Weight: 262.0 lbs, Weight Class: Heavyweight'],
 ['Name: Viviane Araujo Gomes, Nickname: Vivi, DoB: 1986.11.21, Record: 12-6-0 (Win-Loss-Draw), Earnings: $59,400 USD, Height: 5\'4" (163cm), Reach: 68.0" (173cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Josiane Nunes, Nickname: Josi, DoB: 1993.12.23, Record: 10-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'2" (157cm), Reach: 67.0" (170cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Olivier Aubin-Mercier, Nickname: The Canadian Gangster, DoB: 1989.02.23, Record: 21-5-0 (Win-Loss-Draw), Earnings: $154,000 USD, Height: 5\'9" (175cm), Reach: 70.5" (179cm), Weight: 154.8 lbs, Weight Class: Lightweight'],
 ['Name: James Gallagher, Nickname: The Strabanimal, DoB: 1996.10.24, Record: 12-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 67.0" (170cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Fares Ziam, Nickname: Smile Killer, DoB: 1997.03.21, Record: 15-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 75.0" (191cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Joseph Nathan Maness, Nickname: Mayhem, DoB: 1991.06.27, Record: 15-3-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'10" (178cm), Reach: 72.0" (183cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Laird Anderson, Nickname: N/A, DoB: 1998.07.11, Record: 4-0-0 (Win-Loss-Draw), Earnings: $4,000 USD, Height: 5\'7" (170cm), Reach: 65.0" (165cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Nate Landwehr, Nickname: The Train, DoB: 1988.06.07, Record: 18-5-0 (Win-Loss-Draw), Earnings: $286,000 USD, Height: 5\'9" (175cm), Reach: 70.5" (179cm), Weight: 145.3 lbs, Weight Class: Featherweight'],
 ['Name: Олег Попов, Nickname: N/A, DoB: 1992.01.10, Record: 17-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: N/A, Weight: 242.2 lbs, Weight Class: Heavyweight'],
 ['Name: Gillian Robertson, Nickname: The Savage, DoB: 1995.05.17, Record: 13-8-0 (Win-Loss-Draw), Earnings: $170,000 USD, Height: 5\'5" (165cm), Reach: 63.0" (160cm), Weight: 115.8 lbs, Weight Class: Strawweight'],
 ['Name: Impa Kasanganay, Nickname: Tshilobo, DoB: 1994.01.17, Record: 16-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 75.0" (191cm), Weight: 204.8 lbs, Weight Class: Light Heavyweight'],
 ['Name: Marcos Rogério de Lima, Nickname: Pezão, DoB: 1985.05.25, Record: 22-9-1 (Win-Loss-Draw), Earnings: $29,000 USD, Height: 6\'1" (185cm), Reach: 75.0" (191cm), Weight: 261.5 lbs, Weight Class: Heavyweight'],
 ['Name: Joanne Wood, Nickname: JoJo, DoB: 1985.12.23, Record: 17-8-0 (Win-Loss-Draw), Earnings: $162,000 USD, Height: 5\'6" (168cm), Reach: 65.5" (166cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Jordan Newman, Nickname: Not Human, DoB: 1995.07.06, Record: 6-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Jesus Pinedo, Nickname: El Mudo, DoB: 1996.06.22, Record: 23-6-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Andre Petroski, Nickname: N/A, DoB: 1991.06.12, Record: 10-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Aalon Cruz, Nickname: N/A, DoB: 1989.09.20, Record: 11-5-0 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 6\'0" (183cm), Reach: 77.5" (197cm), Weight: 155.6 lbs, Weight Class: Lightweight'],
 ['Name: Jay Jay Wilson, Nickname: The Māori Kid, DoB: 1997.08.02, Record: 10-1-0 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'11" (180cm), Reach: 74.0" (188cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Isaac Dulgarian, Nickname: The Midwest Choppa, DoB: 1996.07.04, Record: 6-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 71.0" (180cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Alex Morono, Nickname: The Great White, DoB: 1990.08.16, Record: 24-10-0, 1 NC (Win-Loss-Draw), Earnings: $154,000 USD, Height: 5\'11" (180cm), Reach: 72.0" (183cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Manel Kape, Nickname: Starboy, DoB: 1993.11.14, Record: 19-6-0 (Win-Loss-Draw), Earnings: $60,000 USD, Height: 5\'5" (165cm), Reach: 68.0" (173cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: James Andrew Miller, Nickname: A-10, DoB: 1983.08.30, Record: 37-18-0, 1 NC (Win-Loss-Draw), Earnings: $1,816,000 USD, Height: 5\'8" (173cm), Reach: 71.0" (180cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Manuel Roberto Torres Cano, Nickname: El Loco, DoB: 1995.03.25, Record: 15-2-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Neil Magny, Nickname: Haitian Sensation, DoB: 1987.08.03, Record: 29-11-0 (Win-Loss-Draw), Earnings: $1,151,000 USD, Height: 6\'3" (191cm), Reach: 80.0" (203cm), Weight: 170.8 lbs, Weight Class: Welterweight'],
 ['Name: Josh O\'Connor, Nickname: N/A, DoB: 1998.11.04, Record: 7-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 70.0" (178cm), Weight: 145.8 lbs, Weight Class: Lightweight'],
 ['Name: Angela Patrice Hill, Nickname: Overkill, DoB: 1985.01.12, Record: 17-13-0 (Win-Loss-Draw), Earnings: $683,000 USD, Height: 5\'3" (160cm), Reach: 64.5" (164cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Marcin Tybura, Nickname: Tybur, DoB: 1985.11.09, Record: 25-8-0 (Win-Loss-Draw), Earnings: $648,000 USD, Height: 6\'3" (191cm), Reach: 78.0" (198cm), Weight: 247.0 lbs, Weight Class: Heavyweight'],
 ['Name: Danny Sabatello, Nickname: The Italian Gangster, DoB: 1993.03.31, Record: 14-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 69.5" (177cm), Weight: 135.8 lbs, Weight Class: Bantamweight'],
 ['Name: Da\'mon Marquel Blackshear, Nickname: Da Monster, DoB: 1994.08.12, Record: 14-6-1 (Win-Loss-Draw), Earnings: $62,000 USD, Height: 5\'10" (178cm), Reach: 72.2" (183cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Mike Breeden, Nickname: Money, DoB: 1989.04.21, Record: 11-6-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 70.0" (178cm), Weight: 159.5 lbs, Weight Class: Lightweight'],
 ['Name: Bryce Logan, Nickname: N/A, DoB: 1989.12.04, Record: 13-8-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 70.0" (178cm), Weight: 155.8 lbs, Weight Class: Lightweight'],
 ['Name: Thalles Renan Ferreira, Nickname: Problema, DoB: 1989.11.23, Record: 13-3-0, 3 NC (Win-Loss-Draw), Earnings: $19,000 USD, Height: 6\'8" (203cm), Reach: 85.0" (216cm), Weight: 263.2 lbs, Weight Class: Heavyweight'],
 ['Name: Thibault Gouti, Nickname: GT, DoB: 1987.02.27, Record: 17-7-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 157.7 lbs, Weight Class: Lightweight'],
 ['Name: Terrance McKinney, Nickname: T.Wrecks, DoB: 1994.09.15, Record: 15-7-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 73.5" (187cm), Weight: 155.3 lbs, Weight Class: Lightweight'],
 ['Name: Магомед Магомедкеримов, Nickname: N/A, DoB: 1990.01.10, Record: 34-6-0 (Win-Loss-Draw), Earnings: $1,275,000 USD, Height: 6\'1" (185cm), Reach: 73.0" (185cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Pat Sabatini, Nickname: N/A, DoB: 1990.11.09, Record: 18-5-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 145.8 lbs, Weight Class: Featherweight'],
 ['Name: Jafel Cavalcante Cordeiro Filho, Nickname: Pastor, DoB: 1993.05.05, Record: 16-3-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 68.5" (174cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: Andrew Martin Dober, Nickname: N/A, DoB: 1988.10.19, Record: 27-13-0, 1 NC (Win-Loss-Draw), Earnings: $708,000 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Demetrious Khrisna Johnson, Nickname: Mighty Mouse, DoB: 1986.08.13, Record: 25-4-1 (Win-Loss-Draw), Earnings: $2,460,000 USD, Height: 5\'3" (161cm), Reach: 66.0" (168cm), Weight: 134.5 lbs, Weight Class: Bantamweight'],
 ['Name: Нуруло Алиев, Nickname: Tajik Eagle, DoB: 2000.12.08, Record: 9-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 72.0" (183cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Miranda Maverick, Nickname: Fear The, DoB: 1997.07.01, Record: 13-5-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'3" (161cm), Reach: 65.5" (166cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Sidney Outlaw, Nickname: Da Gun, DoB: 1992.04.11, Record: 17-6-0 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'9" (175cm), Reach: 76.0" (193cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Cameron Alexander Saaiman, Nickname: MSP, DoB: 2000.12.20, Record: 9-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'8" (173cm), Reach: 67.0" (170cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Bryce Andrew Mitchell, Nickname: Thug Nasty, DoB: 1994.10.04, Record: 16-2-0 (Win-Loss-Draw), Earnings: $152,000 USD, Height: 5\'10" (178cm), Reach: 70.0" (178cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Хизриев Тимур, Nickname: Imam, DoB: 1995.11.19, Record: 15-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Rodolfo Bellato, Nickname: Trator, DoB: 1996.02.05, Record: 12-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'3" (191cm), Reach: 77.5" (197cm), Weight: 205.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Виктория Дудакова, Nickname: N/A, DoB: 1999.01.26, Record: 8-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'5" (165cm), Reach: 67.0" (170cm), Weight: 116.6 lbs, Weight Class: Strawweight'],
 ['Name: Aaron Pico, Nickname: N/A, DoB: 1996.09.23, Record: 13-4-0 (Win-Loss-Draw), Earnings: $380,000 USD, Height: 5\'8" (173cm), Reach: 71.0" (180cm), Weight: 155.7 lbs, Weight Class: Lightweight'],
 ['Name: Levan Chokheli, Nickname: N/A, DoB: 1996.10.02, Record: 13-2-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 169.9 lbs, Weight Class: Welterweight'],
 ['Name: Sara Collins, Nickname: N/A, DoB: 1990.08.22, Record: 5-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 68.5" (174cm), Weight: 145.3 lbs, Weight Class: Featherweight'],
 ['Name: Дмитрий Гриценко, Nickname: N/A, DoB: 1997.04.23, Record: 9-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: N/A, Weight: 158.8 lbs, Weight Class: Lightweight'],
 ['Name: Rakim Cleveland, Nickname: The Boogeyman, DoB: 1989.04.25, Record: 23-17-1, 1 NC (Win-Loss-Draw), Earnings: $80,000 USD, Height: 6\'3" (191cm), Reach: 76.0" (193cm), Weight: 218.8 lbs, Weight Class: Heavyweight'],
 ['Name: Tracy Cortez, Nickname: N/A, DoB: 1993.12.10, Record: 11-1-0 (Win-Loss-Draw), Earnings: $1,000 USD, Height: 5\'5" (165cm), Reach: 65.5" (166cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Cathilee Zingano, Nickname: Alpha, DoB: 1982.07.01, Record: 14-5-0 (Win-Loss-Draw), Earnings: $302,000 USD, Height: 5\'6" (168cm), Reach: 68.0" (173cm), Weight: 144.6 lbs, Weight Class: Featherweight'],
 ['Name: Joshua Hill, Nickname: Gentleman, DoB: 1986.11.24, Record: 22-6-0 (Win-Loss-Draw), Earnings: $24,000 USD, Height: 5\'6" (168cm), Reach: 66.5" (169cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Dustin Stoltzfus, Nickname: N/A, DoB: 1991.11.15, Record: 15-5-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'0" (183cm), Reach: 75.0" (191cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Kennedy Nzechukwu, Nickname: African Savage, DoB: 1992.06.13, Record: 12-5-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 6\'5" (196cm), Reach: 83.0" (211cm), Weight: 205.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Никита Михайлов, Nickname: N/A, DoB: 1998.10.27, Record: 11-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 135.4 lbs, Weight Class: Bantamweight'],
 ['Name: Karl Moore, Nickname: N/A, DoB: 1991.10.24, Record: 12-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 75.0" (191cm), Weight: 204.6 lbs, Weight Class: Light Heavyweight'],
 ['Name: Brady Robert Hiestand, Nickname: Bam Bam, DoB: 1999.04.30, Record: 7-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 71.0" (180cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Асу Алмабаев, Nickname: Zulfikar, DoB: 1994.01.25, Record: 19-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'4" (163cm), Reach: 65.0" (165cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Pascall Saumani Milo, Nickname: The Juggernaut, DoB: 1996.09.21, Record: 5-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'3" (191cm), Reach: 72.0" (183cm), Weight: 249.0 lbs, Weight Class: Heavyweight'],
 ['Name: Касум Касумов, Nickname: N/A, DoB: 1994.07.19, Record: 15-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (172cm), Reach: 66.5" (169cm), Weight: 135.8 lbs, Weight Class: Bantamweight'],
 ['Name: Von Kurt Holobaugh, Nickname: N/A, DoB: 1986.09.08, Record: 20-8-0, 1 NC (Win-Loss-Draw), Earnings: $75,000 USD, Height: 5\'11" (180cm), Reach: 70.0" (178cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Jena Rae Bishop, Nickname: N/A, DoB: 1986.02.22, Record: 7-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'4" (163cm), Reach: 66.5" (169cm), Weight: 125.6 lbs, Weight Class: Flyweight'],
 ['Name: David Grant, Nickname: Dangerous, DoB: 1985.12.18, Record: 13-7-0 (Win-Loss-Draw), Earnings: $258,000 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Anthony J. Smith, Nickname: Lionheart, DoB: 1988.07.26, Record: 38-19-0 (Win-Loss-Draw), Earnings: $906,400 USD, Height: 6\'4" (193cm), Reach: 76.0" (193cm), Weight: 205.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Makhmud Muradov, Nickname: Mach, DoB: 1990.02.08, Record: 26-8-0, 1 NC (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'2" (188cm), Reach: 75.5" (192cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Joshua Van Bawi Thawng, Nickname: The Fearless, DoB: 2001.10.10, Record: 10-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'5" (165cm), Reach: 65.0" (165cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: Nora Cornolle, Nickname: Wonder, DoB: 1989.12.06, Record: 8-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (169cm), Reach: 67.0" (170cm), Weight: 138.5 lbs, Weight Class: Bantamweight'],
 ['Name: Brennan Ward, Nickname: Irish, DoB: 1988.06.28, Record: 17-8-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'10" (178cm), Reach: 72.0" (183cm), Weight: 170.6 lbs, Weight Class: Welterweight'],
 ['Name: Mackenzie Lynne Dern, Nickname: N/A, DoB: 1993.03.24, Record: 13-5-0 (Win-Loss-Draw), Earnings: $366,000 USD, Height: 5\'4" (163cm), Reach: 63.0" (160cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Darren Elkins, Nickname: The Damage, DoB: 1984.05.16, Record: 28-11-0 (Win-Loss-Draw), Earnings: $528,000 USD, Height: 5\'10" (178cm), Reach: 71.0" (180cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Christos Giagos, Nickname: The Spartan, DoB: 1990.01.23, Record: 20-12-0 (Win-Loss-Draw), Earnings: $153,000 USD, Height: 5\'10" (178cm), Reach: 71.5" (182cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Kalinn Fidell Williams, Nickname: The Oxfighter, DoB: 1994.03.30, Record: 15-3-0 (Win-Loss-Draw), Earnings: $200,000 USD, Height: 6\'0" (183cm), Reach: 77.0" (196cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Sumiko Inaba, Nickname: Lady Samurai, DoB: 1991.02.06, Record: 6-1-0 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'4" (163cm), Reach: 64.0" (163cm), Weight: 125.2 lbs, Weight Class: Flyweight'],
 ['Name: Islam Mamedov, Nickname: N/A, DoB: 1989.02.10, Record: 23-3-1 (Win-Loss-Draw), Earnings: $83,000 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 159.4 lbs, Weight Class: Lightweight'],
 ['Name: Ирина Алексеева, Nickname: Russian Ronda, DoB: 1990.06.27, Record: 5-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 66.0" (168cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Devin Terrell Clark, Nickname: Brown Bear, DoB: 1990.04.12, Record: 14-9-0 (Win-Loss-Draw), Earnings: $191,000 USD, Height: 6\'0" (183cm), Reach: 75.0" (191cm), Weight: 204.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Joshua Alvin Culibao, Nickname: Kuya, DoB: 1994.05.24, Record: 11-3-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (177cm), Reach: 73.0" (185cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Carlos Diego Ferreira, Nickname: N/A, DoB: 1985.01.18, Record: 19-5-0 (Win-Loss-Draw), Earnings: $435,000 USD, Height: 5\'9" (175cm), Reach: 74.0" (188cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Anshul Jubli, Nickname: King of Lions, DoB: 1995.01.13, Record: 7-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'0" (183cm), Reach: 69.0" (175cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Michael Shipman, Nickname: Sea Bass, DoB: 1990.06.17, Record: 16-4-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 74.0" (188cm), Weight: 185.9 lbs, Weight Class: Middleweight'],
 ['Name: Gilbert Urbina, Nickname: The RGV Bad Boy, DoB: 1996.04.05, Record: 7-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 75.0" (191cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Ketlen Vieira, Nickname: Fenômeno, DoB: 1991.08.26, Record: 14-3-0 (Win-Loss-Draw), Earnings: $113,000 USD, Height: 5\'8" (173cm), Reach: 68.0" (173cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Sara McMann, Nickname: N/A, DoB: 1980.09.24, Record: 14-7-0 (Win-Loss-Draw), Earnings: $335,000 USD, Height: 5\'6" (168cm), Reach: 66.0" (168cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Ailin Luciana Perez, Nickname: Fiona, DoB: 1994.10.05, Record: 10-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'5" (165cm), Reach: 66.0" (168cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Attila Korkmaz, Nickname: The Fearless, DoB: 1991.07.23, Record: 14-8-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 67.0" (170cm), Weight: 156.1 lbs, Weight Class: Lightweight'],
 ['Name: Kai Kamaka III, Nickname: Fighting Hawaiian, DoB: 1995.01.05, Record: 13-5-1 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 145.8 lbs, Weight Class: Featherweight'],
 ['Name: Darragh Kelly, Nickname: The Moville mauler, DoB: 1998.04.06, Record: 5-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 72.0" (183cm), Weight: 145.2 lbs, Weight Class: Featherweight'],
 ['Name: Ilima-Lei Macfarlane, Nickname: The Ilimanator, DoB: 1990.04.02, Record: 13-3-0 (Win-Loss-Draw), Earnings: $85,000 USD, Height: 5\'4" (163cm), Reach: 66.0" (168cm), Weight: 126.6 lbs, Weight Class: Flyweight'],
 ['Name: андрей корешков Koreshkov Andrey, Nickname: Spartan, DoB: 1990.08.23, Record: 27-5-0 (Win-Loss-Draw), Earnings: $103,000 USD, Height: 6\'1" (185cm), Reach: 75.0" (191cm), Weight: 170.6 lbs, Weight Class: Welterweight'],
 ['Name: Уланбеков Тагир, Nickname: N/A, DoB: 1991.07.08, Record: 15-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Alexander Hernandez, Nickname: The Great Ape, DoB: 1992.10.01, Record: 14-8-0 (Win-Loss-Draw), Earnings: $212,000 USD, Height: 5\'9" (175cm), Reach: 72.0" (183cm), Weight: 150.0 lbs, Weight Class: Featherweight'],
 ['Name: Jefferson Creighton, Nickname: Jazzy, DoB: N/A, Record: 10-2-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 74.0" (188cm), Weight: 170.4 lbs, Weight Class: Welterweight'],
 ['Name: Aiemann Zahabi, Nickname: N/A, DoB: 1987.11.19, Record: 11-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'8" (173cm), Reach: 68.5" (174cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Slim Trabelsi, Nickname: N/A, DoB: 1993.05.09, Record: 7-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 74.0" (188cm), Weight: 239.5 lbs, Weight Class: Heavyweight'],
 ['Name: Мовсар Евлоев, Nickname: N/A, DoB: 1994.02.11, Record: 18-0-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 72.5" (184cm), Weight: 145.8 lbs, Weight Class: Featherweight'],
 ['Name: Viecheslav Borshchev, Nickname: Slava Claus, DoB: 1992.01.08, Record: 7-4-1 (Win-Loss-Draw), Earnings: $162,000 USD, Height: 5\'11" (180cm), Reach: 69.0" (175cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Валентин Молдавский, Valentin Moldavsky, Nickname: N/A, DoB: 1992.02.06, Record: 13-3-0, 1 NC (Win-Loss-Draw), Earnings: $42,733 USD, Height: 6\'1" (185cm), Reach: 75.0" (191cm), Weight: 237.4 lbs, Weight Class: Heavyweight'],
 ['Name: Gokhan Saricam, Nickname: N/A, DoB: 1991.02.26, Record: 9-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 76.0" (193cm), Weight: 246.0 lbs, Weight Class: Heavyweight'],
 ['Name: Romain Debienne, Nickname: N/A, DoB: N/A, Record: 10-5-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (177cm), Reach: 74.0" (188cm), Weight: 170.4 lbs, Weight Class: Welterweight'],
 ['Name: Matthew Burton Brown, Nickname: The Immortal, DoB: 1981.01.10, Record: 24-19-0 (Win-Loss-Draw), Earnings: $1,567,000 USD, Height: 6\'0" (183cm), Reach: 76.0" (193cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Archie Colgan, Nickname: King, DoB: 1995.07.22, Record: 10-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 69.5" (177cm), Weight: 154.9 lbs, Weight Class: Lightweight'],
 ['Name: Jeremy Kennedy, Nickname: JBC, DoB: 1992.09.16, Record: 19-4-0, 1 NC (Win-Loss-Draw), Earnings: $75,000 USD, Height: 5\'11" (180cm), Reach: 71.0" (180cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Jaylon Bates, Nickname: New Breed, DoB: 1997.01.28, Record: 7-1-0 (Win-Loss-Draw), Earnings: $20,000 USD, Height: 5\'10" (178cm), Reach: 73.5" (187cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Morgan Charriere, Nickname: The Last Pirate, DoB: 1995.10.26, Record: 19-10-1 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Josh Emmett, Nickname: CC0, DoB: 1985.03.04, Record: 19-4-0 (Win-Loss-Draw), Earnings: $573,000 USD, Height: 5\'6" (168cm), Reach: 70.0" (178cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Michael Parkin, Nickname: N/A, DoB: 1995.10.02, Record: 9-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'4" (193cm), Reach: 79.0" (201cm), Weight: 262.5 lbs, Weight Class: Heavyweight'],
 ['Name: Robert Font, Nickname: N/A, DoB: 1987.06.25, Record: 20-8-0 (Win-Loss-Draw), Earnings: $543,000 USD, Height: 5\'8" (173cm), Reach: 71.5" (182cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: John Salter, Nickname: N/A, DoB: 1985.03.21, Record: 19-6-0 (Win-Loss-Draw), Earnings: $42,000 USD, Height: 6\'1" (186cm), Reach: 73.5" (187cm), Weight: 185.8 lbs, Weight Class: Middleweight'],
 ['Name: Cody Marcus Brian Durden, Nickname: N/A, DoB: 1991.03.29, Record: 16-5-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 67.0" (170cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Владимир Токов, Nickname: Vova, DoB: 1996.04.14, Record: 9-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 155.2 lbs, Weight Class: Lightweight'],
 ['Name: Volkan Oezdemir, Nickname: No Time, DoB: 1989.09.19, Record: 19-7-0 (Win-Loss-Draw), Earnings: $564,000 USD, Height: 6\'2" (188cm), Reach: 75.0" (191cm), Weight: 205.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Mike Hamel, Nickname: Magic, DoB: 1992.08.12, Record: 11-6-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 68.0" (173cm), Weight: 155.8 lbs, Weight Class: Lightweight'],
 ['Name: Kelvin Gastelum, Nickname: N/A, DoB: 1991.10.24, Record: 18-9-0, 1 NC (Win-Loss-Draw), Earnings: $603,000 USD, Height: 5\'9" (175cm), Reach: 71.5" (182cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Paul Craig, Nickname: Bearjew, DoB: 1987.11.27, Record: 17-8-1 (Win-Loss-Draw), Earnings: $432,000 USD, Height: 6\'3" (192cm), Reach: 76.0" (193cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Victor Westley Henry, Nickname: La Mangosta, DoB: 1987.05.04, Record: 24-6-0, 1 NC (Win-Loss-Draw), Earnings: $40,000 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Don\'Tale O’Neill Mayes, Nickname: Lord Kong, DoB: 1992.01.16, Record: 11-6-0, 1 NC (Win-Loss-Draw), Earnings: $10,000 USD, Height: 6\'6" (198cm), Reach: 81.0" (206cm), Weight: 264.0 lbs, Weight Class: Heavyweight'],
 ['Name: Douglas Lima, Nickname: The Phenom, DoB: 1988.01.05, Record: 33-11-0 (Win-Loss-Draw), Earnings: $230,000 USD, Height: 6\'1" (186cm), Reach: 74.5" (189cm), Weight: 185.8 lbs, Weight Class: Middleweight'],
 ['Name: Holly Rene Holm-Kirkpatrick, Nickname: The Preacher\'s Daughter, DoB: 1981.10.17, Record: 15-7-0 (Win-Loss-Draw), Earnings: $1,600,000 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: David Aliga Onama, Nickname: Silent Assassin, DoB: 1994.06.07, Record: 12-2-0 (Win-Loss-Draw), Earnings: $174,000 USD, Height: 5\'11" (180cm), Reach: 74.0" (188cm), Weight: 148.5 lbs, Weight Class: Featherweight'],
 ['Name: Francisco Prado, Nickname: N/A, DoB: 2002.06.16, Record: 12-2-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'10" (178cm), Reach: 69.3" (176cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Jasmine Jasudavicius, Nickname: N/A, DoB: 1989.03.01, Record: 10-3-0 (Win-Loss-Draw), Earnings: $70,000 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 133.0 lbs, Weight Class: Flyweight'],
 ['Name: Mads Burnell, Nickname: N/A, DoB: 1994.03.06, Record: 18-6-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 68.0" (173cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Andre Muniz de Aguiar, Nickname: Sergipano, DoB: 1990.02.17, Record: 24-6-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'1" (185cm), Reach: 78.0" (198cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Рамазан Курмагомедов, Nickname: N/A, DoB: 1996.08.23, Record: 12-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 73.0" (185cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Suphisara Konklak, Nickname: N/A, DoB: 1996.01.18, Record: 9-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'1" (155cm), Reach: 61.5" (156cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Douglas Silva de Andrade, Nickname: D\'Silva, DoB: 1985.06.22, Record: 29-5-0, 1 NC (Win-Loss-Draw), Earnings: $139,000 USD, Height: 5\'7" (170cm), Reach: 68.5" (174cm), Weight: 139.5 lbs, Weight Class: Bantamweight'],
 ['Name: Mackenzie Stiller, Nickname: MacJudo, DoB: 1999.07.15, Record: 2-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (157cm), Reach: 62.0" (157cm), Weight: 115.8 lbs, Weight Class: Strawweight'],
 ['Name: Uroš Medić, Nickname: The Doctor, DoB: 1993.04.25, Record: 10-2-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'1" (185cm), Reach: 71.0" (180cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Leah McCourt, Nickname: The Curse, DoB: 1992.06.21, Record: 8-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 69.0" (175cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Javid Basharat, Nickname: The Snow Leopard, DoB: 1995.09.07, Record: 14-1-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 69.0" (175cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Bruno Arruda da Silva, Nickname: Blindado, DoB: 1989.07.13, Record: 23-11-0 (Win-Loss-Draw), Earnings: $190,000 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Sedriques Dumas, Nickname: The Reaper, DoB: 1995.08.06, Record: 9-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 79.0" (201cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Modestas Bukauskas, Nickname: The Baltic Gladiator, DoB: 1994.02.10, Record: 15-6-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'3" (191cm), Reach: 78.0" (198cm), Weight: 205.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Gregory Babene, Nickname: Blade, DoB: 1984.01.26, Record: 23-12-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 185.7 lbs, Weight Class: Middleweight'],
 ['Name: Jennifer Maia, Nickname: N/A, DoB: 1988.10.06, Record: 21-10-1 (Win-Loss-Draw), Earnings: $119,500 USD, Height: 5\'4" (163cm), Reach: 64.0" (163cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Grant Neal, Nickname: The Truth, DoB: 1995.10.05, Record: 9-2-0 (Win-Loss-Draw), Earnings: $16,000 USD, Height: 5\'11" (180cm), Reach: 74.0" (188cm), Weight: 205.4 lbs, Weight Class: Light Heavyweight'],
 ['Name: Augusto Sakai, Nickname: N/A, DoB: 1991.05.19, Record: 16-5-1 (Win-Loss-Draw), Earnings: $124,000 USD, Height: 6\'3" (191cm), Reach: 77.0" (196cm), Weight: 265.0 lbs, Weight Class: Heavyweight'],
 ['Name: Justine Kish, Nickname: The Machine, DoB: 1988.04.13, Record: 9-6-0 (Win-Loss-Draw), Earnings: $34,000 USD, Height: 5\'5" (165cm), Reach: 64.5" (164cm), Weight: 125.6 lbs, Weight Class: Flyweight'],
 ['Name: Miesha Tate, Nickname: Cupcake, DoB: 1986.08.18, Record: 20-9-0 (Win-Loss-Draw), Earnings: $1,136,500 USD, Height: 5\'6" (168cm), Reach: 66.5" (169cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Ion Cutelaba, Nickname: The Hulk, DoB: 1993.12.14, Record: 17-10-1, 1 NC (Win-Loss-Draw), Earnings: $137,000 USD, Height: 6\'1" (185cm), Reach: 75.0" (191cm), Weight: 205.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Jack Shore, Nickname: Tank, DoB: 1995.02.06, Record: 17-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'8" (173cm), Reach: 71.0" (180cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Bruno Gustavo Aparecido da Silva, Nickname: Bulldog, DoB: 1990.03.16, Record: 13-5-2, 1 NC (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'4" (163cm), Reach: 66.0" (168cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: Jéssica Andrade, Nickname: Bate Estaca, DoB: 1991.09.25, Record: 26-12-0 (Win-Loss-Draw), Earnings: $742,000 USD, Height: 5\'1" (156cm), Reach: 62.0" (157cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Matt Frevola, Nickname: The Steamrolla, DoB: 1990.06.11, Record: 11-4-1 (Win-Loss-Draw), Earnings: $116,000 USD, Height: 5\'9" (175cm), Reach: 71.0" (180cm), Weight: 155.6 lbs, Weight Class: Lightweight'],
 ['Name: Jared Gordon, Nickname: Flash, DoB: 1988.09.06, Record: 20-6-0, 1 NC (Win-Loss-Draw), Earnings: $180,000 USD, Height: 5\'9" (175cm), Reach: 68.0" (173cm), Weight: 155.6 lbs, Weight Class: Lightweight'],
 ['Name: Jeremiah C. Wells, Nickname: N/A, DoB: 1986.10.30, Record: 12-4-1 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'9" (175cm), Reach: 74.0" (188cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Carlos Vergara, Nickname: N/A, DoB: 1991.06.18, Record: 12-5-1 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'6" (168cm), Reach: 68.0" (173cm), Weight: 127.0 lbs, Weight Class: Flyweight'],
 ['Name: Linton Vassell, Nickname: The Big Swarm, DoB: 1983.06.03, Record: 24-9-0, 1 NC (Win-Loss-Draw), Earnings: $202,000 USD, Height: 6\'4" (193cm), Reach: 80.5" (204cm), Weight: 240.8 lbs, Weight Class: Heavyweight'],
 ['Name: Нурсултан Рузибоев, Nickname: Black, DoB: 1993.11.19, Record: 34-9-2, 2 NC (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'5" (196cm), Reach: 76.0" (193cm), Weight: 170.3 lbs, Weight Class: Welterweight'],
 ['Name: Sean Brady, Nickname: N/A, DoB: 1992.11.23, Record: 16-1-0 (Win-Loss-Draw), Earnings: $124,000 USD, Height: 5\'10" (178cm), Reach: 72.5" (184cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Газиев Шамиль Хасбулаевич, Nickname: N/A, DoB: 1990.02.10, Record: 12-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'4" (193cm), Reach: 78.5" (199cm), Weight: 261.0 lbs, Weight Class: Heavyweight'],
 ['Name: Denise Pacheco Gomes, Nickname: Dee, DoB: 1999.12.30, Record: 8-3-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'2" (157cm), Reach: 63.0" (160cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Michael Jerome Reece-Page, Nickname: Venom, DoB: 1987.04.07, Record: 22-2-0 (Win-Loss-Draw), Earnings: $40,000 USD, Height: 6\'3" (191cm), Reach: 79.0" (201cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Trey Waters, Nickname: The Truth, DoB: 1995.02.21, Record: 9-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'5" (196cm), Reach: 77.0" (196cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Enrique Jhonatan Barzola Acosta, Nickname: El Fuerte, DoB: 1989.04.28, Record: 20-8-2 (Win-Loss-Draw), Earnings: $42,000 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Corey Anderson, Nickname: Overtime, DoB: 1989.09.22, Record: 18-6-0, 1 NC (Win-Loss-Draw), Earnings: $796,000 USD, Height: 6\'3" (191cm), Reach: 77.5" (197cm), Weight: 204.6 lbs, Weight Class: Light Heavyweight'],
 ['Name: Aaron Jeffery, Nickname: N/A, DoB: 1992.11.14, Record: 14-5-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 73.0" (185cm), Weight: 185.2 lbs, Weight Class: Middleweight'],
 ['Name: Tainara B de Almeida Lisboa, Nickname: Thai Panther, DoB: 1991.03.02, Record: 7-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 67.0" (170cm), Weight: 133.5 lbs, Weight Class: Bantamweight'],
 ['Name: Lucas Brennan, Nickname: Skywalker, DoB: 2000.06.03, Record: 9-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 71.0" (180cm), Weight: 145.6 lbs, Weight Class: Featherweight'],
 ['Name: Luca Poclit, Nickname: N/A, DoB: 1989.09.21, Record: 10-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 169.8 lbs, Weight Class: Welterweight'],
 ['Name: 박현성, Park Hyun Sung, Nickname: Peace of Mind, DoB: 1995.11.04, Record: 9-0-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (171cm), Reach: 66.0" (168cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Zak Cummings, Nickname: N/A, DoB: 1984.08.02, Record: 25-7-0 (Win-Loss-Draw), Earnings: $129,000 USD, Height: 6\'0" (183cm), Reach: 75.0" (191cm), Weight: 206.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Drakkar Klose, Nickname: N/A, DoB: 1988.03.09, Record: 15-2-1 (Win-Loss-Draw), Earnings: $356,000 USD, Height: 5\'9" (175cm), Reach: 70.0" (178cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Cody Brundage, Nickname: N/A, DoB: 1994.05.16, Record: 10-6-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'0" (183cm), Reach: 72.0" (183cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Amanda Ribas, Nickname: N/A, DoB: 1993.08.26, Record: 12-5-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'3" (160cm), Reach: 66.0" (168cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: William Daniel Quarantillo, Nickname: N/A, DoB: 1988.12.08, Record: 18-6-0 (Win-Loss-Draw), Earnings: $124,000 USD, Height: 5\'10" (178cm), Reach: 70.0" (178cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Rodolfo Vieira Srour, Nickname: The Black Belt Hunter, DoB: 1989.09.25, Record: 10-2-0 (Win-Loss-Draw), Earnings: $178,000 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Irene Aldana Robles, Nickname: N/A, DoB: 1988.03.26, Record: 15-7-0 (Win-Loss-Draw), Earnings: $360,400 USD, Height: 5\'9" (175cm), Reach: 68.5" (174cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Jalin Turner, Nickname: The Tarantula, DoB: 1995.05.18, Record: 14-8-0 (Win-Loss-Draw), Earnings: $129,000 USD, Height: 6\'3" (191cm), Reach: 75.5" (192cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Rafael Garcia Jr., Nickname: Gifted, DoB: 1994.08.05, Record: 16-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Luke Trainer, Nickname: The Gent, DoB: 1996.02.27, Record: 8-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'6" (198cm), Reach: 81.0" (206cm), Weight: 203.8 lbs, Weight Class: Light Heavyweight'],
 ['Name: Veronica Hardy, Nickname: N/A, DoB: 1995.10.30, Record: 9-4-1 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'4" (163cm), Reach: 64.0" (163cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Магомед Магомедов, Nickname: Tiger, DoB: 1991.12.25, Record: 20-4-0 (Win-Loss-Draw), Earnings: $15,000 USD, Height: 5\'7" (171cm), Reach: 69.0" (175cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Joshua Fremd, Nickname: N/A, DoB: 1994.01.15, Record: 11-5-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'4" (193cm), Reach: 76.0" (193cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Robert Glenn Lawler, Nickname: Ruthless, DoB: 1982.03.20, Record: 30-16-0, 1 NC (Win-Loss-Draw), Earnings: $3,759,940 USD, Height: 5\'11" (181cm), Reach: 74.0" (188cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Joel Alvarez Gonzalez, Nickname: El Fenomeno, DoB: 1993.03.02, Record: 20-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 77.0" (196cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Kyler John Phillips, Nickname: The Matrix, DoB: 1995.06.12, Record: 12-2-0 (Win-Loss-Draw), Earnings: $170,000 USD, Height: 5\'8" (173cm), Reach: 72.0" (183cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Patrick Pimblett, Nickname: The Baddy, DoB: 1995.01.03, Record: 21-3-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Brunno Klynton Angulski Santiago Ferreira, Nickname: The Hulk, DoB: 1992.11.04, Record: 11-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'10" (179cm), Reach: 72.0" (183cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Timothy Elliott, Nickname: N/A, DoB: 1986.12.24, Record: 20-13-1 (Win-Loss-Draw), Earnings: $498,000 USD, Height: 5\'7" (170cm), Reach: 66.0" (168cm), Weight: 136.0 lbs, Weight Class: Flyweight'],
 ['Name: Jonny Parsons, Nickname: The Sluggernaut, DoB: 1991.09.06, Record: 9-3-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'9" (175cm), Reach: 69.0" (175cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Pedro Munhoz-Lopes, Nickname: The Young Punisher, DoB: 1986.09.07, Record: 20-9-0, 2 NC (Win-Loss-Draw), Earnings: $696,000 USD, Height: 5\'6" (168cm), Reach: 65.0" (165cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Leandro Higo Lucena Pinto, Nickname: Pitbull, DoB: 1989.01.19, Record: 23-6-0 (Win-Loss-Draw), Earnings: $14,000 USD, Height: 5\'8" (173cm), Reach: 70.5" (179cm), Weight: 145.1 lbs, Weight Class: Featherweight'],
 ['Name: Allan Alves do Nascimento, Nickname: Puro Osso, DoB: 1991.09.11, Record: 20-6-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'8" (173cm), Reach: 69.5" (177cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: Mohammed Usman, Nickname: The Motor, DoB: 1989.04.01, Record: 10-3-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'2" (188cm), Reach: 79.0" (201cm), Weight: 237.0 lbs, Weight Class: Heavyweight'],
 ['Name: Dustin Anthony Jacoby, Nickname: The Hanyak, DoB: 1988.04.04, Record: 19-8-1 (Win-Loss-Draw), Earnings: $121,000 USD, Height: 6\'3" (191cm), Reach: 76.0" (193cm), Weight: 204.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Derrick Lewis, Nickname: The Black Beast, DoB: 1985.02.07, Record: 28-12-0, 1 NC (Win-Loss-Draw), Earnings: $1,112,000 USD, Height: 6\'3" (191cm), Reach: 79.0" (201cm), Weight: 264.0 lbs, Weight Class: Heavyweight'],
 ['Name: Tim Wilde, Nickname: The Experiment, DoB: 1987.06.30, Record: 17-5-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 74.0" (188cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Gabriel de Souza Bonfim, Nickname: Marretinha, DoB: 1997.08.20, Record: 15-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'1" (185cm), Reach: 72.5" (184cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Diego Lopes, Nickname: N/A, DoB: 1994.12.30, Record: 24-6-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'11" (180cm), Reach: 72.5" (184cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Ciryl Romain Gane, Nickname: Bon Gamin, DoB: 1990.04.12, Record: 12-2-0 (Win-Loss-Draw), Earnings: $700,000 USD, Height: 6\'4" (193cm), Reach: 81.0" (206cm), Weight: 250.0 lbs, Weight Class: Heavyweight'],
 ['Name: Mario Alberto Bautista, Nickname: N/A, DoB: 1993.07.01, Record: 14-2-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'9" (175cm), Reach: 69.0" (175cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Rodrigo Nascimento Ferreira, Nickname: Zé Colmeia, DoB: 1992.11.26, Record: 11-2-0, 1 NC (Win-Loss-Draw), Earnings: $20,000 USD, Height: 6\'2" (188cm), Reach: 80.0" (203cm), Weight: 265.0 lbs, Weight Class: Heavyweight'],
 ['Name: Bryan Battle, Nickname: The Butcher, DoB: 1994.09.21, Record: 10-2-0, 1 NC (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'1" (185cm), Reach: 77.0" (196cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Вадим Немков, Vadim Nemkov, Nickname: N/A, DoB: 1992.06.20, Record: 18-2-0, 1 NC (Win-Loss-Draw), Earnings: $330,000 USD, Height: 6\'0" (183cm), Reach: 76.0" (193cm), Weight: 238.1 lbs, Weight Class: Heavyweight'],
 ['Name: Fabian Edwards, Nickname: The Assassin, DoB: 1993.03.24, Record: 13-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 79.5" (202cm), Weight: 185.6 lbs, Weight Class: Middleweight'],
 ['Name: Chase Alan Hooper, Nickname: The Dream, DoB: 1999.09.13, Record: 14-3-1 (Win-Loss-Draw), Earnings: $125,000 USD, Height: 6\'1" (185cm), Reach: 74.0" (188cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Daniel Alexander Ige, Nickname: 50k, DoB: 1991.08.06, Record: 18-7-0 (Win-Loss-Draw), Earnings: $340,000 USD, Height: 5\'7" (170cm), Reach: 71.0" (180cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: John Castañeda, Nickname: Sexi Mexi, DoB: 1991.12.18, Record: 21-6-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'6" (168cm), Reach: 71.0" (180cm), Weight: 137.6 lbs, Weight Class: Bantamweight'],
 ['Name: Daniel Olvera Zellhuber, Nickname: Golden Boy, DoB: 1999.07.07, Record: 15-1-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'1" (185cm), Reach: 77.0" (196cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Kevin Jousset, Nickname: Air, DoB: 1993.05.02, Record: 10-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'2" (188cm), Reach: 75.0" (191cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Raquel Pennington, Nickname: Rocky, DoB: 1988.09.05, Record: 16-8-0 (Win-Loss-Draw), Earnings: $391,000 USD, Height: 5\'7" (170cm), Reach: 67.5" (171cm), Weight: 134.8 lbs, Weight Class: Bantamweight'],
 ['Name: Cody Garbrandt, Nickname: No Love, DoB: 1991.07.07, Record: 14-6-0 (Win-Loss-Draw), Earnings: $1,128,000 USD, Height: 5\'8" (173cm), Reach: 65.5" (166cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Muhammad Naimov, Nickname: Hillman, DoB: 1994.08.07, Record: 11-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'9" (175cm), Reach: 70.0" (178cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Kyle Nelson, Nickname: The Monster, DoB: 1991.04.20, Record: 16-5-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 71.0" (180cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Cristiane Justino Venancio, Nickname: N/A, DoB: 1985.07.09, Record: 27-2-0, 1 NC (Win-Loss-Draw), Earnings: $2,388,000 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 144.6 lbs, Weight Class: Featherweight'],
 ['Name: Iasmin Lucindo Bezerra, Nickname: N/A, DoB: 2002.01.08, Record: 16-5-0 (Win-Loss-Draw), Earnings: $62,000 USD, Height: 5\'3" (160cm), Reach: 66.0" (168cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Chris Duncan, Nickname: The Problem, DoB: 1993.05.10, Record: 11-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 71.5" (182cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Brandon Royval, Nickname: Raw Dawg, DoB: 1992.08.16, Record: 16-7-0 (Win-Loss-Draw), Earnings: $224,000 USD, Height: 5\'9" (175cm), Reach: 70.5" (179cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Julija Stoliarenko, Nickname: N/A, DoB: 1993.04.08, Record: 11-8-1 (Win-Loss-Draw), Earnings: $60,000 USD, Height: 5\'7" (170cm), Reach: 69.0" (175cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: William Gomis, Nickname: Jaguar, DoB: 1997.06.13, Record: 13-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Luana do Valle Lopes Gonzales Santos, Nickname: N/A, DoB: 2000.04.16, Record: 7-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'6" (168cm), Reach: 67.0" (170cm), Weight: 139.0 lbs, Weight Class: Bantamweight'],
 ['Name: Stephen Vincenzo Erceg, Nickname: AstroBoy, DoB: 1995.07.27, Record: 12-2-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'8" (173cm), Reach: 68.5" (174cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Marlon Andres Vera Delgado, Nickname: Chito, DoB: 1992.12.02, Record: 23-9-1 (Win-Loss-Draw), Earnings: $975,000 USD, Height: 5\'8" (173cm), Reach: 70.5" (179cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Montel Jackson, Nickname: Quik, DoB: 1992.04.24, Record: 13-2-0 (Win-Loss-Draw), Earnings: $110,000 USD, Height: 5\'10" (178cm), Reach: 75.5" (192cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Logan Storley, Nickname: Storm, DoB: 1992.09.08, Record: 15-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 71.5" (182cm), Weight: 170.6 lbs, Weight Class: Welterweight'],
 ['Name: Natália Miranda Cristina da Silva, Nickname: N/A, DoB: 1997.02.03, Record: 17-5-1 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'4" (163cm), Reach: 65.0" (165cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: Michel Pereira, Nickname: Demolidor, DoB: 1993.10.06, Record: 31-11-0, 2 NC (Win-Loss-Draw), Earnings: $400,000 USD, Height: 6\'1" (186cm), Reach: 73.0" (185cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: 中村倫也, Nakamura Rinya, Nickname: Hybrid, DoB: 1995.03.23, Record: 9-0-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 68.0" (173cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Alexander Shabliy, Nickname: Peresvet, DoB: 1993.04.18, Record: 24-3-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (176cm), Reach: 70.5" (179cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Daniel James, Nickname: The American Predator, DoB: 1981.12.31, Record: 15-7-1, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'6" (198cm), Reach: 80.0" (203cm), Weight: 267.0 lbs, Weight Class: Heavyweight'],
 ['Name: Gunnar Lúðvík Nelson, Nickname: Gunni, DoB: 1988.07.28, Record: 19-5-1 (Win-Loss-Draw), Earnings: $383,000 USD, Height: 5\'11" (181cm), Reach: 72.0" (183cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Grant Dawson, Nickname: KGD, DoB: 1994.02.20, Record: 21-2-1 (Win-Loss-Draw), Earnings: $123,800 USD, Height: 5\'10" (178cm), Reach: 72.0" (183cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Giga Chikadze, Nickname: Ninja, DoB: 1988.08.25, Record: 15-3-0 (Win-Loss-Draw), Earnings: $253,200 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Karolina Kowalkiewicz, Nickname: N/A, DoB: 1985.10.15, Record: 16-8-0 (Win-Loss-Draw), Earnings: $117,000 USD, Height: 5\'3" (160cm), Reach: 64.0" (163cm), Weight: 116.0 lbs, Weight Class: Strawweight'],
 ['Name: Deiveson Alcantara, Nickname: Deus da Guerra, DoB: 1987.12.18, Record: 23-3-1 (Win-Loss-Draw), Earnings: $505,000 USD, Height: 5\'5" (165cm), Reach: 68.0" (173cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Tabatha Ricci Fabri Salto, Nickname: Baby Shark, DoB: 1995.02.21, Record: 10-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'1" (155cm), Reach: 61.0" (155cm), Weight: 115.0 lbs, Weight Class: Strawweight'],
 ['Name: Marvin Vettori, Nickname: The Italian Dream, DoB: 1993.09.20, Record: 19-6-1 (Win-Loss-Draw), Earnings: $393,000 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Kevin Luke Swanson, Nickname: Killer, DoB: 1983.11.02, Record: 29-13-0 (Win-Loss-Draw), Earnings: $900,440 USD, Height: 5\'8" (173cm), Reach: 70.0" (178cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Karine Silva de Almeida, Nickname: Killer, DoB: 1993.12.02, Record: 18-4-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'5" (165cm), Reach: 67.0" (170cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: 박준용, Park Jun Yong, Nickname: The Iron Turtle, DoB: 1991.02.27, Record: 17-6-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'10" (178cm), Reach: 73.0" (185cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Farid Basharat, Nickname: Ferocious, DoB: 1997.08.02, Record: 12-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'8" (173cm), Reach: 71.0" (180cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Karl Williams, Nickname: N/A, DoB: 1990.01.06, Record: 10-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'3" (191cm), Reach: 79.0" (201cm), Weight: 246.5 lbs, Weight Class: Heavyweight'],
 ['Name: Ринат Фахретдинов, Nickname: Gladiator, DoB: 1991.09.28, Record: 22-2-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Ismael Bonfim, Nickname: Marreta, DoB: 1995.12.28, Record: 20-4-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'8" (173cm), Reach: 71.0" (180cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Marcus Julius McGhee, Nickname: The Maniac, DoB: 1990.05.07, Record: 9-1-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 135.5 lbs, Weight Class: Bantamweight'],
 ['Name: Мыктыбек Оролбай уулу, Nickname: N/A, DoB: 1998.02.10, Record: 13-1-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'10" (178cm), Reach: 74.0" (188cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Ľudovít Klein, Nickname: Mr. Highlight, DoB: 1995.02.22, Record: 21-4-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 72.5" (184cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Yair Rodriguez Portillo, Nickname: El Pantera, DoB: 1992.10.06, Record: 18-5-0, 1 NC (Win-Loss-Draw), Earnings: $592,000 USD, Height: 5\'11" (180cm), Reach: 71.0" (180cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Alonzo Theodoro Menifield, Nickname: Atomic, DoB: 1987.10.18, Record: 15-4-1 (Win-Loss-Draw), Earnings: $84,000 USD, Height: 6\'0" (183cm), Reach: 76.0" (193cm), Weight: 205.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Patrick Mix, Nickname: No Love, DoB: 1993.08.16, Record: 20-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 71.5" (182cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Nikita Andreyvich Krylov, Nickname: The Miner, DoB: 1992.07.03, Record: 30-9-0 (Win-Loss-Draw), Earnings: $266,000 USD, Height: 6\'3" (191cm), Reach: 77.5" (197cm), Weight: 213.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Vitor Petrino Malaquias Salvo, Nickname: Icão, DoB: 1997.08.28, Record: 11-1-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'2" (188cm), Reach: 77.5" (197cm), Weight: 206.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Caio Borralho, Nickname: The Natural, DoB: 1993.01.16, Record: 16-1-0, 1 NC (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'1" (185cm), Reach: 75.0" (191cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Bo Dean Nickal, Nickname: N/A, DoB: 1996.01.14, Record: 6-0-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'1" (185cm), Reach: 76.0" (193cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Joseph Oliver Pyfer, Nickname: Bodybagz, DoB: 1996.09.17, Record: 12-3-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'2" (188cm), Reach: 75.0" (191cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Jared Christopher Cannonier, Nickname: Killa Gorilla, DoB: 1984.03.16, Record: 17-6-0 (Win-Loss-Draw), Earnings: $432,000 USD, Height: 5\'11" (180cm), Reach: 77.5" (197cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Johnny Eblen, Nickname: Pressure, DoB: 1991.12.13, Record: 15-0-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 6\'1" (185cm), Reach: 74.0" (188cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: 平良達郎, Taira Tatsuro, Nickname: N/A, DoB: 2000.01.27, Record: 15-0-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 125.5 lbs, Weight Class: Flyweight'],
 ['Name: Marc-André Barriault, Nickname: Powerbar, DoB: 1990.02.18, Record: 16-7-0, 1 NC (Win-Loss-Draw), Earnings: $78,000 USD, Height: 6\'1" (185cm), Reach: 74.0" (188cm), Weight: 184.5 lbs, Weight Class: Middleweight'],
 ['Name: Mayra Bueno Silva, Nickname: Sheetara, DoB: 1991.08.22, Record: 10-3-1, 1 NC (Win-Loss-Draw), Earnings: $150,000 USD, Height: 5\'6" (168cm), Reach: 66.5" (169cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Joaquin Buckley, Nickname: New Mansa, DoB: 1994.04.27, Record: 19-6-0 (Win-Loss-Draw), Earnings: $200,000 USD, Height: 5\'10" (178cm), Reach: 76.0" (193cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Bobby Ryan Green, Nickname: King, DoB: 1986.09.09, Record: 32-15-1, 1 NC (Win-Loss-Draw), Earnings: $689,000 USD, Height: 5\'10" (178cm), Reach: 71.0" (180cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Michael Morales Hurtado, Nickname: N/A, DoB: 1999.06.24, Record: 16-0-0 (Win-Loss-Draw), Earnings: $20,000 USD, Height: 6\'0" (183cm), Reach: 79.0" (201cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Алискеров Икрам, Nickname: N/A, DoB: 1992.12.07, Record: 15-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'0" (183cm), Reach: 76.0" (193cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Jose Mariscal, Nickname: Machine Gun, DoB: 1992.10.27, Record: 16-6-0, 1 NC (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 69.0" (175cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Kevin Holland, Nickname: Trailblazer, DoB: 1992.11.05, Record: 26-11-0, 1 NC (Win-Loss-Draw), Earnings: $475,000 USD, Height: 6\'3" (191cm), Reach: 81.0" (206cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Justin Tafa, Nickname: Bad Man, DoB: 1993.12.13, Record: 7-4-0, 1 NC (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'0" (183cm), Reach: 74.0" (188cm), Weight: 264.5 lbs, Weight Class: Heavyweight'],
 ['Name: Ярослав Амосов, Nickname: Dynamo, DoB: 1993.09.09, Record: 27-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 75.0" (191cm), Weight: 169.6 lbs, Weight Class: Welterweight'],
 ['Name: Usman Nurmagomedov, Nickname: N/A, DoB: 1998.04.17, Record: 17-0-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'11" (180cm), Reach: 72.0" (183cm), Weight: 154.8 lbs, Weight Class: Lightweight'],
 ['Name: Сергей Павлович, Nickname: N/A, DoB: 1992.05.13, Record: 18-2-0 (Win-Loss-Draw), Earnings: $200,000 USD, Height: 6\'3" (191cm), Reach: 84.0" (213cm), Weight: 259.8 lbs, Weight Class: Heavyweight'],
 ['Name: Nazim Sadykhov, Nickname: Black Wolf, DoB: 1994.05.16, Record: 9-1-1 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'10" (178cm), Reach: 69.0" (175cm), Weight: 155.6 lbs, Weight Class: Lightweight'],
 ['Name: Sergio Pettis, Nickname: The Phenom, DoB: 1993.08.18, Record: 23-6-0 (Win-Loss-Draw), Earnings: $412,000 USD, Height: 5\'6" (168cm), Reach: 69.0" (175cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Vicente Luque, Nickname: The Silent Assassin, DoB: 1991.11.27, Record: 22-10-1 (Win-Loss-Draw), Earnings: $706,000 USD, Height: 5\'11" (180cm), Reach: 76.0" (193cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Aljamain Sterling, Nickname: Funk Master, DoB: 1989.07.31, Record: 24-4-0 (Win-Loss-Draw), Earnings: $518,000 USD, Height: 5\'7" (171cm), Reach: 71.0" (180cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Luis Philipe Lins, Nickname: Monstro, DoB: 1985.08.17, Record: 18-5-0 (Win-Loss-Draw), Earnings: $1,160,000 USD, Height: 6\'2" (188cm), Reach: 78.0" (198cm), Weight: 206.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Daniel Aaron Marcos Moreno, Nickname: Soncora, DoB: 1993.03.07, Record: 15-0-0, 1 NC (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (170cm), Reach: 69.0" (175cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Liz Carmouche, Nickname: Girl-Rilla, DoB: 1984.02.19, Record: 21-7-0 (Win-Loss-Draw), Earnings: $126,500 USD, Height: 5\'6" (168cm), Reach: 66.0" (168cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Norma Dumont Viana Ferreira, Nickname: The Immortal, DoB: 1990.10.01, Record: 11-2-0 (Win-Loss-Draw), Earnings: $10,000 USD, Height: 5\'7" (169cm), Reach: 67.0" (170cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Carlston Lindsay Harris, Nickname: Moçambique, DoB: 1987.07.09, Record: 19-6-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'0" (183cm), Reach: 76.0" (193cm), Weight: 169.5 lbs, Weight Class: Welterweight'],
 ['Name: 闫晓楠, Yan Xiaonan, Nickname: Nine, DoB: 1989.06.16, Record: 17-4-0, 1 NC (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'5" (166cm), Reach: 63.0" (160cm), Weight: 115.0 lbs, Weight Class: Strawweight'],
 ['Name: William Algeo, Nickname: Señor Perfecto, DoB: 1989.06.09, Record: 18-8-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Dan Hooker, Nickname: The Hangman, DoB: 1990.02.13, Record: 23-12-0 (Win-Loss-Draw), Earnings: $543,000 USD, Height: 6\'0" (183cm), Reach: 75.5" (192cm), Weight: 155.5 lbs, Weight Class: Lightweight'],
 ['Name: Lerone Murphy, Nickname: The Miracle, DoB: 1991.07.22, Record: 14-0-1 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'9" (175cm), Reach: 73.5" (187cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Amir Albazi, Nickname: The Prince, DoB: 1993.10.27, Record: 17-1-0 (Win-Loss-Draw), Earnings: $32,000 USD, Height: 5\'5" (165cm), Reach: 68.0" (173cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Manon Fiorot, Nickname: The Beast, DoB: 1990.02.17, Record: 12-1-0 (Win-Loss-Draw), Earnings: $80,000 USD, Height: 5\'7" (169cm), Reach: 66.0" (168cm), Weight: 125.3 lbs, Weight Class: Flyweight'],
 ['Name: Joanderson Sá Brito, Nickname: Tubarão, DoB: 1995.02.11, Record: 17-3-1 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'8" (173cm), Reach: 72.0" (183cm), Weight: 146.0 lbs, Weight Class: Featherweight'],
 ['Name: Muhammad Mokaev, Nickname: The Punisher, DoB: 2000.07.30, Record: 11-0-0, 1 NC (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Christian Rodriguez, Nickname: CeeRod, DoB: 1997.12.17, Record: 11-1-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'7" (170cm), Reach: 71.0" (180cm), Weight: 145.0 lbs, Weight Class: Featherweight'],
 ['Name: Charles Jourdain, Nickname: Air, DoB: 1995.11.27, Record: 15-7-1 (Win-Loss-Draw), Earnings: $116,000 USD, Height: 5\'9" (175cm), Reach: 69.0" (175cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Anthony Hernandez, Nickname: Fluffy, DoB: 1993.10.18, Record: 12-2-0, 1 NC (Win-Loss-Draw), Earnings: $112,000 USD, Height: 6\'0" (183cm), Reach: 75.0" (191cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Nasrat Mohammad Haqparast, Nickname: N/A, DoB: 1995.08.22, Record: 16-5-0 (Win-Loss-Draw), Earnings: $175,000 USD, Height: 5\'10" (178cm), Reach: 72.0" (183cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Steve Garcia Jr., Nickname: Mean Machine, DoB: 1992.05.22, Record: 15-5-0 (Win-Loss-Draw), Earnings: $62,000 USD, Height: 6\'0" (183cm), Reach: 73.0" (185cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Ryan D. Bader, Nickname: Darth, DoB: 1983.06.07, Record: 31-8-0, 1 NC (Win-Loss-Draw), Earnings: $634,000 USD, Height: 6\'2" (188cm), Reach: 74.0" (188cm), Weight: 231.1 lbs, Weight Class: Heavyweight'],
 ['Name: Азамат Мурзаканов, Nickname: The Professional, DoB: 1989.04.12, Record: 13-0-0 (Win-Loss-Draw), Earnings: $74,000 USD, Height: 5\'10" (179cm), Reach: 71.5" (182cm), Weight: 205.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Нурмагомедов Омар, Nickname: N/A, DoB: 1996.01.03, Record: 17-0-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'8" (172cm), Reach: 69.0" (175cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Сергій Співак, Nickname: Polar Bear, DoB: 1995.01.24, Record: 16-4-0 (Win-Loss-Draw), Earnings: $64,000 USD, Height: 6\'3" (190cm), Reach: 78.0" (198cm), Weight: 256.0 lbs, Weight Class: Heavyweight'],
 ['Name: Virna Jandiroba, Nickname: Carcará, DoB: 1988.05.30, Record: 20-3-0 (Win-Loss-Draw), Earnings: $62,000 USD, Height: 5\'3" (160cm), Reach: 64.0" (163cm), Weight: 115.0 lbs, Weight Class: Strawweight'],
 ['Name: Sean Strickland, Nickname: Tarzan, DoB: 1991.02.27, Record: 29-6-0 (Win-Loss-Draw), Earnings: $344,000 USD, Height: 6\'1" (185cm), Reach: 76.0" (193cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Ислам Махачев, Nickname: N/A, DoB: 1991.10.27, Record: 26-1-0 (Win-Loss-Draw), Earnings: $352,000 USD, Height: 5\'10" (178cm), Reach: 70.5" (179cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Leon Edwards, Nickname: Rocky, DoB: 1991.08.25, Record: 22-3-0, 1 NC (Win-Loss-Draw), Earnings: $465,000 USD, Height: 6\'2" (188cm), Reach: 74.0" (188cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Thomas Aspinall, Nickname: N/A, DoB: 1993.04.11, Record: 14-3-0 (Win-Loss-Draw), Earnings: $300,000 USD, Height: 6\'5" (196cm), Reach: 78.0" (198cm), Weight: 261.6 lbs, Weight Class: Heavyweight'],
 ['Name: Karen Alexa Grasso Montes, Nickname: N/A, DoB: 1993.08.09, Record: 16-3-1 (Win-Loss-Draw), Earnings: $191,000 USD, Height: 5\'5" (165cm), Reach: 66.0" (168cm), Weight: 124.5 lbs, Weight Class: Flyweight'],
 ['Name: Alexandre Pantoja Passidomo, Nickname: The Cannibal, DoB: 1990.04.16, Record: 28-5-0 (Win-Loss-Draw), Earnings: $300,000 USD, Height: 5\'5" (166cm), Reach: 68.0" (173cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Dricus Du Plessis, Nickname: Stillknocks, DoB: 1994.01.14, Record: 21-2-0 (Win-Loss-Draw), Earnings: $227,000 USD, Height: 6\'1" (185cm), Reach: 76.0" (193cm), Weight: 184.0 lbs, Weight Class: Middleweight'],
 ['Name: Jonathan Dwight Jones, Nickname: Bones, DoB: 1987.07.19, Record: 27-1-0, 1 NC (Win-Loss-Draw), Earnings: $4,568,000 USD, Height: 6\'4" (194cm), Reach: 84.5" (215cm), Weight: 248.0 lbs, Weight Class: Heavyweight'],
 ['Name: Justin Gaethje, Nickname: The Highlight, DoB: 1988.11.14, Record: 25-5-0 (Win-Loss-Draw), Earnings: $1,662,000 USD, Height: 5\'11" (181cm), Reach: 70.0" (178cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Alex Sandro Silva Pereira, Nickname: Poatan, DoB: 1987.07.07, Record: 10-2-0 (Win-Loss-Draw), Earnings: $200,000 USD, Height: 6\'4" (194cm), Reach: 80.0" (203cm), Weight: 205.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Sean O\'Malley, Nickname: Suga, DoB: 1994.10.24, Record: 18-1-0, 1 NC (Win-Loss-Draw), Earnings: $699,000 USD, Height: 5\'11" (180cm), Reach: 72.0" (183cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Jailton Almeida, Nickname: Malhadinho, DoB: 1991.06.26, Record: 21-3-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 6\'3" (191cm), Reach: 79.0" (201cm), Weight: 241.0 lbs, Weight Class: Heavyweight'],
 ['Name: Шавкат Рахмонов, Nickname: Nomad, DoB: 1994.10.23, Record: 18-0-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 6\'1" (185cm), Reach: 77.0" (196cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Saint Denis, Nickname: God of War, DoB: 1995.12.18, Record: 13-2-0, 1 NC (Win-Loss-Draw), Earnings: $200,000 USD, Height: 5\'11" (180cm), Reach: 73.0" (185cm), Weight: 155.0 lbs, Weight Class: Lightweight'],
 ['Name: Jamahal Hill, Nickname: Sweet Dreams, DoB: 1991.05.19, Record: 12-2-0, 1 NC (Win-Loss-Draw), Earnings: $224,000 USD, Height: 6\'4" (193cm), Reach: 79.0" (201cm), Weight: 205.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Brendan Cody Allen, Nickname: All In, DoB: 1995.12.28, Record: 24-5-0 (Win-Loss-Draw), Earnings: $202,000 USD, Height: 6\'2" (188cm), Reach: 75.0" (191cm), Weight: 186.0 lbs, Weight Class: Middleweight'],
 ['Name: Ian Machado Garry, Nickname: The Future, DoB: 1997.11.17, Record: 14-0-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 6\'3" (191cm), Reach: 74.0" (188cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Erin Blanchfield, Nickname: Cold Blooded, DoB: 1999.05.04, Record: 12-2-0 (Win-Loss-Draw), Earnings: $50,000 USD, Height: 5\'4" (163cm), Reach: 68.0" (173cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Jerome-Max Kelii Holloway, Nickname: Blessed, DoB: 1991.12.04, Record: 26-7-0 (Win-Loss-Draw), Earnings: $2,200,000 USD, Height: 5\'11" (180cm), Reach: 69.0" (175cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Cory Sandhagen, Nickname: Sandman, DoB: 1992.04.20, Record: 17-4-0 (Win-Loss-Draw), Earnings: $500,000 USD, Height: 5\'11" (180cm), Reach: 70.0" (178cm), Weight: 139.5 lbs, Weight Class: Bantamweight'],
 ['Name: Giacomo Della Maddalena, Nickname: N/A, DoB: 1996.09.10, Record: 17-2-0 (Win-Loss-Draw), Earnings: $270,000 USD, Height: 5\'11" (180cm), Reach: 73.0" (185cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Merab Dvalishvili, Nickname: The Machine, DoB: 1991.01.10, Record: 17-4-0 (Win-Loss-Draw), Earnings: $354,000 USD, Height: 5\'6" (168cm), Reach: 68.0" (173cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Tatiana Yadira Suarez Padilla, Nickname: N/A, DoB: 1990.12.19, Record: 10-0-0 (Win-Loss-Draw), Earnings: $230,000 USD, Height: 5\'5" (165cm), Reach: 66.0" (168cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Mateusz Gamrot, Nickname: Gamer, DoB: 1990.12.11, Record: 24-2-0, 1 NC (Win-Loss-Draw), Earnings: $202,000 USD, Height: 5\'10" (177cm), Reach: 70.5" (179cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Walker Johnny Silva Barra de Souza, Nickname: N/A, DoB: 1992.03.30, Record: 21-8-0, 1 NC (Win-Loss-Draw), Earnings: $290,000 USD, Height: 6\'6" (198cm), Reach: 82.3" (209cm), Weight: 205.5 lbs, Weight Class: Light Heavyweight'],
 ['Name: Alexander Volkanovski, Nickname: The Great, DoB: 1988.09.29, Record: 26-4-0 (Win-Loss-Draw), Earnings: $575,000 USD, Height: 5\'6" (168cm), Reach: 71.5" (182cm), Weight: 144.5 lbs, Weight Class: Featherweight'],
 ['Name: Ilia Topuria, Nickname: El Matador, DoB: 1997.01.21, Record: 15-0-0 (Win-Loss-Draw), Earnings: $200,000 USD, Height: 5\'7" (170cm), Reach: 69.0" (175cm), Weight: 144.5 lbs, Weight Class: Featherweight'],
 ['Name: 宋亚东, Song Yadong, Nickname: Kung Fu Kid, DoB: 1997.12.02, Record: 21-8-1, 1 NC (Win-Loss-Draw), Earnings: $486,000 USD, Height: 5\'8" (173cm), Reach: 67.0" (170cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Charles Oliveira, Nickname: do Bronx, DoB: 1989.10.17, Record: 34-10-0, 1 NC (Win-Loss-Draw), Earnings: $1,347,600 USD, Height: 5\'10" (178cm), Reach: 74.0" (188cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Александр Волков, Nickname: Drago, DoB: 1988.10.24, Record: 37-10-0 (Win-Loss-Draw), Earnings: $315,000 USD, Height: 6\'7" (201cm), Reach: 80.0" (203cm), Weight: 263.0 lbs, Weight Class: Heavyweight'],
 ['Name: Amanda Nunes, Nickname: The Lioness, DoB: 1988.05.30, Record: 23-5-0 (Win-Loss-Draw), Earnings: $2,428,500 USD, Height: 5\'8" (173cm), Reach: 69.0" (175cm), Weight: 135.0 lbs, Weight Class: Bantamweight'],
 ['Name: Maria Lupita Godinez, Nickname: Loopy, DoB: 1993.09.06, Record: 12-4-0 (Win-Loss-Draw), Earnings: $95,000 USD, Height: 5\'2" (157cm), Reach: 61.0" (155cm), Weight: 115.0 lbs, Weight Class: Strawweight'],
 ['Name: Nicolas Dalby, Nickname: Danish Dynamite, DoB: 1984.11.16, Record: 23-4-1, 2 NC (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'11" (180cm), Reach: 74.5" (189cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Edson Mendes Barboza, Nickname: Junior, DoB: 1986.01.21, Record: 24-12-0 (Win-Loss-Draw), Earnings: $1,306,000 USD, Height: 5\'11" (181cm), Reach: 75.0" (191cm), Weight: 145.5 lbs, Weight Class: Featherweight'],
 ['Name: Brandon Moreno, Nickname: The Assassin Baby, DoB: 1993.12.07, Record: 21-8-2 (Win-Loss-Draw), Earnings: $656,000 USD, Height: 5\'7" (170cm), Reach: 70.0" (178cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Maycee Barber, Nickname: The Future, DoB: 1998.05.18, Record: 14-2-0 (Win-Loss-Draw), Earnings: $79,000 USD, Height: 5\'5" (165cm), Reach: 65.0" (165cm), Weight: 125.0 lbs, Weight Class: Flyweight'],
 ['Name: Belal Muhammad, Nickname: Remember the Name, DoB: 1988.07.09, Record: 23-3-0, 1 NC (Win-Loss-Draw), Earnings: $372,000 USD, Height: 5\'11" (180cm), Reach: 72.0" (183cm), Weight: 170.0 lbs, Weight Class: Welterweight'],
 ['Name: Khalil Ibn Rountree Jr., Nickname: The War Horse, DoB: 1990.02.06, Record: 13-5-0, 1 NC (Win-Loss-Draw), Earnings: $342,000 USD, Height: 6\'1" (185cm), Reach: 76.5" (194cm), Weight: 204.0 lbs, Weight Class: Light Heavyweight'],
 ['Name: Роман Копылов, Nickname: N/A, DoB: 1991.05.04, Record: 13-3-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'0" (183cm), Reach: 75.0" (191cm), Weight: 185.0 lbs, Weight Class: Middleweight'],
 ['Name: Jonathan Martinez, Nickname: Dragon, DoB: 1994.04.20, Record: 19-5-0 (Win-Loss-Draw), Earnings: $178,000 USD, Height: 5\'8" (173cm), Reach: 69.5" (177cm), Weight: 136.0 lbs, Weight Class: Bantamweight'],
 ['Name: Арман Царукян, Nickname: Ahalkalakets, DoB: 1996.10.11, Record: 22-3-0 (Win-Loss-Draw), Earnings: $250,000 USD, Height: 5\'7" (170cm), Reach: 72.5" (184cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: Elves Brenner Oliveira, Nickname: N/A, DoB: 1997.09.27, Record: 16-4-0 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 5\'10" (178cm), Reach: 72.0" (183cm), Weight: 156.0 lbs, Weight Class: Lightweight'],
 ['Name: 张伟丽, Zhang Weli, Nickname: Magnum, DoB: 1989.08.13, Record: 25-3-0 (Win-Loss-Draw), Earnings: $514,000 USD, Height: 5\'4" (163cm), Reach: 63.0" (160cm), Weight: 115.0 lbs, Weight Class: Strawweight'],
 ['Name: Gilbert Burns, Nickname: Durinho, DoB: 1986.07.20, Record: 22-7-0 (Win-Loss-Draw), Earnings: $643,000 USD, Height: 5\'10" (178cm), Reach: 71.0" (180cm), Weight: 171.0 lbs, Weight Class: Welterweight'],
 ['Name: Khamzat Girikhanov, Nickname: Borz, DoB: 1994.05.01, Record: 13-0-0 (Win-Loss-Draw), Earnings: $250,000 USD, Height: 6\'2" (188cm), Reach: 75.0" (191cm), Weight: 185.5 lbs, Weight Class: Middleweight'],
 ['Name: Michael Malott, Nickname: Proper, DoB: 1991.11.07, Record: 10-2-1 (Win-Loss-Draw), Earnings: $100,000 USD, Height: 6\'1" (185cm), Reach: 73.0" (185cm), Weight: 170.5 lbs, Weight Class: Welterweight'],
 ['Name: Israel Mobolaji Odunayo Adesanya, Nickname: The Last Stylebender, DoB: 1989.07.22, Record: 24-3-0 (Win-Loss-Draw), Earnings: $1,468,000 USD, Height: 6\'4" (193cm), Reach: 80.0" (203cm), Weight: 184.8 lbs, Weight Class: Middleweight'],
 ['Name: Mateusz Tadeusz Rebecki, Nickname: Rebeasti, DoB: 1992.10.03, Record: 19-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'7" (169cm), Reach: 66.0" (168cm), Weight: 155.3 lbs, Weight Class: Lightweight'],
 ['Name: Ariane Lipski, Nickname: Queen of Violence, DoB: 1994.01.26, Record: 17-9-0 (Win-Loss-Draw), Earnings: $142,000 USD, Height: 5\'6" (168cm), Reach: 67.0" (170cm), Weight: 126.0 lbs, Weight Class: Flyweight'],
 ['Name: Carlos Sao Murry Ulberg, Nickname: Black Jag, DoB: 1990.11.17, Record: 10-1-0 (Win-Loss-Draw), Earnings: $150,000 USD, Height: 6\'4" (193cm), Reach: 77.0" (196cm), Weight: 205.0 lbs, Weight Class: Light Heavyweight']]

In [2]:
df

[['Name: Екатерина Шакалова, Nickname: N/A, DoB: 1997.08.31, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (158cm), Reach: 62.6" (159cm), Weight: 144.2 lbs, Weight Class: Featherweight'],
 ['Name: Emily Marisa Ducote, Nickname: Gordinha, DoB: 1994.01.01, Record: 13-9-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'2" (157cm), Reach: 63.0" (160cm), Weight: 115.5 lbs, Weight Class: Strawweight'],
 ['Name: Fabacary Diatta, Nickname: N/A, DoB: 1996.10.16, Record: 9-1-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: 70.5" (179cm), Weight: 145.4 lbs, Weight Class: Featherweight'],
 ['Name: Dumitru Girlean, Nickname: N/A, DoB: 1994.11.07, Record: 8-2-0 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'9" (175cm), Reach: N/A, Weight: 154.3 lbs, Weight Class: Lightweight'],
 ['Name: Ana Talita de Oliveira Alencar, Nickname: Problem Child, DoB: 1990.10.17, Record: 5-0-1 (Win-Loss-Draw), Earnings: $0 USD, Height: 5\'1" (155cm), Reach: 58.5" (149cm), Weight: 116.0 lb

In [4]:
test_df = df[:10]

In [13]:
import re

In [15]:
match = re.search(r'Name: ([^,]+)', test_df[0])

if match:
    name = match.group(1)
    print(name)

TypeError: expected string or bytes-like object, got 'list'

In [12]:
import pandas as pd

test_df[0][0].find(
    'Name: ',','
)


TypeError: slice indices must be integers or None or have an __index__ method

In [8]:
fighter_df = fighter_list

In [38]:
import pandas as pd

# Function to parse each entry into a dictionary
def parse_row(row):
    info = {}
    for part in row[0].split(','):
        key, value = part.split(':', 1)
        info[key.strip()] = value.strip()
    return info

# Creating a list of dictionaries
parsed_data = [parse_row(row) for row in fighter_df]

# Converting to DataFrame
df = pd.DataFrame(parsed_data)

# Display the DataFrame
print(df.head())  # Show only the first few rows for brevity


ValueError: not enough values to unpack (expected 2, got 1)

In [9]:
import pandas as pd

# Function to parse each entry into a dictionary
def parse_row(row):
    info = {}
    parts = row[0].split(',')
    for part in parts:
        if ':' in part:  # Check if ':' is present
            key, value = part.split(':', 1)
            info[key.strip()] = value.strip()
        else:
            print(f"Skipping malformed part: {part}")  # Optionally print/log the problematic parts
    return info

# Creating a list of dictionaries
parsed_data = [parse_row(row) for row in fighter_df]

# Converting to DataFrame
df = pd.DataFrame(parsed_data)

# Display the DataFrame
print(df.head())  # Show only the first few rows for brevity


Skipping malformed part: 000 USD
Skipping malformed part:  2 NC (Win-Loss-Draw)
Skipping malformed part:  Tamaru Toki
Skipping malformed part: 000 USD
Skipping malformed part:  Yoshinari Nadaka
Skipping malformed part: 000 USD
Skipping malformed part: 400 USD
Skipping malformed part: 000 USD
Skipping malformed part: 000 USD
Skipping malformed part:  Choi Doo Ho
Skipping malformed part: 000 USD
Skipping malformed part: 000 USD
Skipping malformed part:  Jung Chan Sung
Skipping malformed part: 000 USD
Skipping malformed part:  Song Kenan
Skipping malformed part: 000 USD
Skipping malformed part:  1 NC (Win-Loss-Draw)
Skipping malformed part: 000 USD
Skipping malformed part: 794
Skipping malformed part: 000 USD
Skipping malformed part:  1 NC (Win-Loss-Draw)
Skipping malformed part: 000 USD
Skipping malformed part:  1 NC (Win-Loss-Draw)
Skipping malformed part: 000 USD
Skipping malformed part: 000 USD
Skipping malformed part: 000 USD
Skipping malformed part: 000 USD
Skipping malformed part: 

In [10]:
df.head()

,Name,Nickname,DoB,Record,Earnings,Height,Reach,Weight,Weight Class
0,Екатерина Шакалова,N/A,1997.08.31,8-2-0 (Win-Loss-Draw),$0 USD,"5'2"" (158cm)","62.6"" (159cm)",144.2 lbs,Featherweight
1,Emily Marisa Ducote,Gordinha,1994.01.01,13-9-0 (Win-Loss-Draw),$0 USD,"5'2"" (157cm)","63.0"" (160cm)",115.5 lbs,Strawweight
2,Fabacary Diatta,N/A,1996.10.16,9-1-0 (Win-Loss-Draw),$0 USD,"5'9"" (175cm)","70.5"" (179cm)",145.4 lbs,Featherweight
3,Dumitru Girlean,N/A,1994.11.07,8-2-0 (Win-Loss-Draw),$0 USD,"5'9"" (175cm)",N/A,154.3 lbs,Lightweight
4,Ana Talita de Oliveira Alencar,Problem Child,1990.10.17,5-0-1 (Win-Loss-Draw),$0 USD,"5'1"" (155cm)","58.5"" (149cm)",116.0 lbs,Strawweight


In [56]:
df.columns = ['name','nickname','date_of_birth','record','earnings_$','height_cm','reach_cm','weight_kg','weight_class']

In [57]:
df

,name,nickname,date_of_birth,record,earnings_$,height_cm,reach_cm,weight_kg,weight_class
0,Екатерина Шакалова,N/A,1997.08.31,8-2-0 (Win-Loss-Draw),$0 USD,"5'2"" (158cm)","62.6"" (159cm)",144.2 lbs,Featherweight
1,Emily Marisa Ducote,Gordinha,1994.01.01,13-9-0 (Win-Loss-Draw),$0 USD,"5'2"" (157cm)","63.0"" (160cm)",115.5 lbs,Strawweight
